### This notebook includes annual data for:  
+ Unemployment Rate  
+ GDP  
+ Educational Awards  
+ Occupational Employment  
+ Industry Employment

In [5]:
#packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [ ]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

Unemployment Rate:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "LAUS" dataset, annual
+ Select the following dimensions: Region, Year 
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)     
+ Under "Year" choose your range (most recently I've done 2000 - recent)
+ Select the value "Labor Force", "Employed", and Unemployed". Unemployment rate is available but we're calculating it here in case we ever want to aggregate anything before calculating a percentage. It's been checked that this is correct and aligns with JobsEQ's data.    
+ Swap rows & columns if needed so "Regions" data is at the top, and "Year" is at the side. 

When you've downloaded this csv, open it and delete the top line, replace "region" with "NAME" as a double header, unmerge the time and make sure it's copy-pasted across the relevant columns and keep the headers for different datapoints. Delete the data notes at the bottom, and select all of the numerical data and make them "General" format. Save as 'JobsEQ_LAUSAnnual_GeosExceptPlaces.csv'

In [3]:
#import LAUS data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_LAUSAnnual_GeosExceptPlaces.csv')
data.head(3)

,Year,USA,USA.1,USA.2,Tennessee (47),Tennessee (47).1,Tennessee (47).2,GNRC,GNRC.1,GNRC.2,MPO,MPO.1,MPO.2,GNRC Region,GNRC Region.1,GNRC Region.2,"Cheatham County, Tennessee (47021)","Cheatham County, Tennessee (47021).1","Cheatham County, Tennessee (47021).2","Davidson County, Tennessee (47037)","Davidson County, Tennessee (47037).1","Davidson County, Tennessee (47037).2","Dickson County, Tennessee (47043)","Dickson County, Tennessee (47043).1","Dickson County, Tennessee (47043).2","Houston County, Tennessee (47083)","Houston County, Tennessee (47083).1","Houston County, Tennessee (47083).2","Humphreys County, Tennessee (47085)","Humphreys County, Tennessee (47085).1","Humphreys County, Tennessee (47085).2","Maury County, Tennessee (47119)","Maury County, Tennessee (47119).1","Maury County, Tennessee (47119).2","Montgomery County, Tennessee (47125)","Montgomery County, Tennessee (47125).1","Montgomery County, Tennessee (47125).2","Robertson County, Tennessee (47147)","Robertson County, Tennessee (47147).1","Robertson County, Tennessee (47147).2","Rutherford County, Tennessee (47149)","Rutherford County, Tennessee (47149).1","Rutherford County, Tennessee (47149).2","Stewart County, Tennessee (47161)","Stewart County, Tennessee (47161).1","Stewart County, Tennessee (47161).2","Sumner County, Tennessee (47165)","Sumner County, Tennessee (47165).1","Sumner County, Tennessee (47165).2","Trousdale County, Tennessee (47169)","Trousdale County, Tennessee (47169).1","Trousdale County, Tennessee (47169).2","Williamson County, Tennessee (47187)","Williamson County, Tennessee (47187).1","Williamson County, Tennessee (47187).2","Wilson County, Tennessee (47189)","Wilson County, Tennessee (47189).1","Wilson County, Tennessee (47189).2"
0,Year,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed
1,2000,143893565,138065883,5827681,2849026,2738785,110241,753959,730195,23763,668134,647631,20502,790723,765674,25048,19513,18982,531,309162,299570,9592,22119,21339,780,3673,3461,212,8882,8445,437,36764,35479,1284,59350,57148,2202,29566,28535,1031,102558,99470,3088,5499,5236,262,70402,68164,2237,3549,3429,119,69927,68208,1719,49752,48203,1549
2,2001,145059940,138106699,6953241,2857659,2728621,129038,758444,730269,28174,671818,647629,24188,795016,765363,29652,19598,18957,641,306620,295925,10695,22203,21250,953,3619,3367,251,8746,8198,548,36572,35093,1478,59841,57262,2579,29998,28749,1249,105013,101216,3796,5592,5297,294,71680,68523,3157,3595,3400,195,71479,69508,1970,50453,48612,1841


In [4]:
#set index to the extraneous "Time" header and transpose, then rename to "Datapoint" and reset index without dropping
data = data.set_index('Year').transpose()
data = data.rename(columns = {'Year': 'Datapoint'}).reset_index(drop = False)

In [5]:
data.head()

Year,index,Datapoint,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,USA,Labor Force,143893565,145059940,146160666,147005871,148071836,149975300,152127646,153578409,155106956,155126715,155523182,155740916,156232399,156528594,157068244,157960658,159791241,161841127,163121138,164906736,162407439,163401845,165971079
1,USA.1,Employed,138065883,138106699,137615237,138129110,139818438,142222730,145000041,146388401,146047744,140696560,140454859,141714416,143548583,144904563,147293813,149540786,151934228,154721774,156709675,158806258,149294638,154624087,159884645
2,USA.2,Unemployed,5827681,6953241,8545428,8876760,8253397,7752570,7127604,7190008,9059212,14430154,15068323,14026499,12683815,11624031,9774431,8419872,7857013,7119352,6411462,6100477,13112801,8777758,6086434
3,Tennessee (47),Labor Force,2849026,2857659,2898947,2904565,2897792,2898489,3014748,3043258,3060778,3032658,3093118,3117492,3096322,3078892,3043573,3074215,3136398,3197031,3257052,3335269,3303230,3323545,3352029
4,Tennessee (47).1,Employed,2738785,2728621,2750423,2744350,2742910,2738223,2856078,2901946,2862093,2725134,2789055,2833278,2849727,2840127,2842540,2902683,2987678,3077514,3143791,3223663,3056709,3174135,3238558


In [6]:
#make sure there's no head or tail on the strings
data['index'] = data['index'].str.strip()
data['Datapoint'] = data['Datapoint'].str.strip()

In [7]:
#split and expand the geography names on the "." (where there were multiple from duplicates because of how we imported it) and take what was before as the name
boop = data['index'].str.split(pat = ".", expand = True)
data['NAME'] = boop[0]

In [8]:
#drop the original column, set the index to NAME and transpose to rename, use custom module, then transpose back and reset index without dropping
data = data.drop(columns = 'index').set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [9]:
data.head()

Year,NAME,Datapoint,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,United States,Labor Force,143893565,145059940,146160666,147005871,148071836,149975300,152127646,153578409,155106956,155126715,155523182,155740916,156232399,156528594,157068244,157960658,159791241,161841127,163121138,164906736,162407439,163401845,165971079
1,United States,Employed,138065883,138106699,137615237,138129110,139818438,142222730,145000041,146388401,146047744,140696560,140454859,141714416,143548583,144904563,147293813,149540786,151934228,154721774,156709675,158806258,149294638,154624087,159884645
2,United States,Unemployed,5827681,6953241,8545428,8876760,8253397,7752570,7127604,7190008,9059212,14430154,15068323,14026499,12683815,11624031,9774431,8419872,7857013,7119352,6411462,6100477,13112801,8777758,6086434
3,Tennessee,Labor Force,2849026,2857659,2898947,2904565,2897792,2898489,3014748,3043258,3060778,3032658,3093118,3117492,3096322,3078892,3043573,3074215,3136398,3197031,3257052,3335269,3303230,3323545,3352029
4,Tennessee,Employed,2738785,2728621,2750423,2744350,2742910,2738223,2856078,2901946,2862093,2725134,2789055,2833278,2849727,2840127,2842540,2902683,2987678,3077514,3143791,3223663,3056709,3174135,3238558


In [10]:
#go from wide to long format with .melt() naming the columns as single column "Time", keeping "Datapoint" and "NAME" as is, and calling the number "Value"
cols = data.columns
data = data.melt(var_name = 'Year', id_vars = ['NAME', 'Datapoint'], value_vars = cols, value_name = 'Value')
data.head(2)

,NAME,Datapoint,Year,Value
0,United States,Labor Force,2000,143893565
1,United States,Employed,2000,138065883


In [11]:
#set index and transpose on datapoint to rename the original categories from JobsEQ, then transpose and reset index again without dropping
data = data.set_index('Datapoint').transpose()
data = data.rename(columns = {'Labor Force': 'LF:Labor Force', 'Employed': 'LF:Employed', 
                              'Unemployed': 'LF:Unemployed'})
data = data.transpose().reset_index(drop = False)

In [12]:
data.head()

,Datapoint,NAME,Year,Value
0,LF:Labor Force,United States,2000,143893565
1,LF:Employed,United States,2000,138065883
2,LF:Unemployed,United States,2000,5827681
3,LF:Labor Force,Tennessee,2000,2849026
4,LF:Employed,Tennessee,2000,2738785


In [13]:
#long to wide
data = pd.pivot(data, index = ['NAME', 'Year'], columns = 'Datapoint', values = 'Value')

In [14]:
cols = list(data.columns)
data[cols] = data[cols].astype(float)

In [15]:
data['LF:Unemployment %'] = (data['LF:Unemployed']/data['LF:Labor Force']) * 100

In [16]:
data = data.reset_index(drop = False)

In [17]:
data.head()

Datapoint,NAME,Year,LF:Employed,LF:Labor Force,LF:Unemployed,LF:Unemployment %
0,"Cheatham County, Tennessee",2000,18982.0,19513.0,531.0,2.721263
1,"Cheatham County, Tennessee",2001,18957.0,19598.0,641.0,3.270742
2,"Cheatham County, Tennessee",2002,19042.0,19816.0,774.0,3.905935
3,"Cheatham County, Tennessee",2003,18934.0,19756.0,821.0,4.155700
4,"Cheatham County, Tennessee",2004,19153.0,20054.0,900.0,4.487883


In [18]:
#make sure year is formatted as an integer
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [19]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Datapoint                  LF:Employed                                                                                                      \
Year                              2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
NAME                                                                                                                                         
Cheatham County, Tennessee     18982.0   18957.0   19042.0   18934.0   19153.0   19307.0   20020.0   19629.0   19332.0   18409.0   18379.0   
Davidson County, Tennessee    299570.0  295925.0  292120.0  289174.0  288420.0  287713.0  296913.0  310958.0  307246.0  293470.0  314127.0   

Datapoint                                                                                                                                           \
Year                            2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
NAME                                                                                                                                                 
Cheatham County, Tennessee   18533.0   18667.0   18692.0   18764.0   18952.0   19462.0   20229.0   20789.0   21368.0   20160.0   21330.0   22102.0   
Davidson County, Tennessee  322412.0  330508.0  334535.0  339009.0  349099.0  360600.0  374318.0  384281.0  393665.0  366705.0  387755.0  402976.0   

Datapoint                  LF:Labor Force                                                                                                      \
Year                                 2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
NAME                                                                                                                                            
Cheatham County, Tennessee        19513.0   19598.0   19816.0   19756.0   20054.0   20186.0   20880.0   20405.0   20431.0   20262.0   20220.0   
Davidson County, Tennessee       309162.0  306620.0  305054.0  302814.0  301955.0  301278.0  310198.0  323213.0  324564.0  321138.0  342567.0   

Datapoint                                                                                                                                           \
Year                            2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
NAME                                                                                                                                                 
Cheatham County, Tennessee   20273.0   20144.0   20007.0   19857.0   19907.0   20271.0   20868.0   21377.0   21948.0   21366.0   22015.0   22683.0   
Davidson County, Tennessee  349109.0  352776.0  355490.0  356981.0  364943.0  374154.0  385090.0  394750.0  403955.0  399262.0  404746.0  414876.0   

Datapoint                  LF:Unemployed                                                                                                     \
Year                                2000     2001     2002     2003     2004     2005     2006     2007     2008     2009     2010     2011   
NAME                                                                                                                                          
Cheatham County, Tennessee         531.0    641.0    774.0    821.0    900.0    878.0    860.0    775.0   1099.0   1852.0   1840.0   1740.0   
Davidson County, Tennessee        9592.0  10695.0  12934.0  13639.0  13534.0  13564.0  13284.0  12254.0  17317.0  27668.0  28440.0  26697.0   

Datapoint                                                                                                                     LF:Unemployment %  \
Year                           2012     2013     2014     2015     2016     2017     2018     2019     2020     2021     2022              2000   
NAME                                               

In [20]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

LF:Employed                                                                                                      \
                                  2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
                                  None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                         
Cheatham County, Tennessee     18982.0   18957.0   19042.0   18934.0   19153.0   19307.0   20020.0   19629.0   19332.0   18409.0   18379.0   
Davidson County, Tennessee    299570.0  295925.0  292120.0  289174.0  288420.0  287713.0  296913.0  310958.0  307246.0  293470.0  314127.0   
Dickson County, Tennessee      21339.0   21250.0   21309.0   21381.0   21431.0   21503.0   22390.0   22268.0   22011.0   20858.0   21488.0   

                                                                                                                                                    \
                                2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
                                None      None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                                 
Cheatham County, Tennessee   18533.0   18667.0   18692.0   18764.0   18952.0   19462.0   20229.0   20789.0   21368.0   20160.0   21330.0   22102.0   
Davidson County, Tennessee  322412.0  330508.0  334535.0  339009.0  349099.0  360600.0  374318.0  384281.0  393665.0  366705.0  387755.0  402976.0   
Dickson County, Tennessee    21913.0   22121.0   22048.0   22142.0   22841.0   23714.0   24697.0   25576.0   26392.0   24786.0   26173.0   27207.0   

                           LF:Labor Force                                                                                                      \
                                     2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
                                     None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                            
Cheatham County, Tennessee        19513.0   19598.0   19816.0   19756.0   20054.0   20186.0   20880.0   20405.0   20431.0   20262.0   20220.0   
Davidson County, Tennessee       309162.0  306620.0  305054.0  302814.0  301955.0  301278.0  310198.0  323213.0  324564.0  321138.0  342567.0   
Dickson County, Tennessee         22119.0   22203.0   22375.0   22487.0   22592.0   22620.0   23491.0   23225.0   23622.0   23357.0   23980.0   

                                                                                                                                                    \
                                2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
                                None      None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                                 
Cheatham County, Tennessee   20273.0   20144.0   20007.0   19857.0   19907.0   20271.0   20868.0   21377.0   21948.0   21366.0   22015.0   22683.0   
Davidson County, Tennessee  349109.0  352776.0  355490.0  356981.0  364943.0  374154.0  385090.0  394750.0  403955.0  399262.0  404746.0  414876.0   
Dickson County, Tennessee    24270.0   24166.0   23876.0   23627.

In [21]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
first_level

['LF:Employed', 'LF:Labor Force', 'LF:Unemployed', 'LF:Unemployment %']

In [22]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [23]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [24]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ LAUS'

In [25]:
#it's weird to look at percent change of a percent so dropping the % change as to not confuse anyone later
#also renaming the real change, this is one of the only places it's applicable because normally we wouldn't run change metrics on a percent but want to for ue
data = data.drop(columns = ['LF:Unemployment % % Change'])
data = data.rename(columns = {'LF:Unemployment % Change': 'LF:Unemployment % Real Change'})

In [26]:
#final check
data.head()

,NAME,Year,Time Frame,LF:Employed,LF:Employed % Change,LF:Employed Change,LF:Labor Force,LF:Labor Force % Change,LF:Labor Force Change,LF:Unemployed,LF:Unemployed % Change,LF:Unemployed Change,LF:Unemployment %,LF:Unemployment % Real Change,GEO_ID,Source
0,"Cheatham County, Tennessee",2000,None,18982.0,NaN,NaN,19513.0,NaN,NaN,531.0,NaN,NaN,2.721263,NaN,0500000US47021,JobsEQ LAUS
1,"Cheatham County, Tennessee",2001,None,18957.0,NaN,NaN,19598.0,NaN,NaN,641.0,NaN,NaN,3.270742,NaN,0500000US47021,JobsEQ LAUS
2,"Cheatham County, Tennessee",2002,None,19042.0,NaN,NaN,19816.0,NaN,NaN,774.0,NaN,NaN,3.905935,NaN,0500000US47021,JobsEQ LAUS
3,"Cheatham County, Tennessee",2003,None,18934.0,NaN,NaN,19756.0,NaN,NaN,821.0,NaN,NaN,4.155700,NaN,0500000US47021,JobsEQ LAUS
4,"Cheatham County, Tennessee",2004,None,19153.0,NaN,NaN,20054.0,NaN,NaN,900.0,NaN,NaN,4.487883,NaN,0500000US47021,JobsEQ LAUS


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5244 entries, 0 to 5243
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   NAME                           5244 non-null   object 
 1   Year                           5244 non-null   object 
 2   Time Frame                     5244 non-null   object 
 3   LF:Employed                    437 non-null    float64
 4   LF:Employed % Change           4807 non-null   float64
 5   LF:Employed Change             4807 non-null   float64
 6   LF:Labor Force                 437 non-null    float64
 7   LF:Labor Force % Change        4807 non-null   float64
 8   LF:Labor Force Change          4807 non-null   float64
 9   LF:Unemployed                  437 non-null    float64
 10  LF:Unemployed % Change         4807 non-null   float64
 11  LF:Unemployed Change           4807 non-null   float64
 12  LF:Unemployment %              437 non-null    f

In [28]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_LAUS_Annual_Change', conn, if_exists = 'replace', index = False)

5244

GDP:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "GDP" dataset  
+ Select the following dimensions: Region, Year 
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)     
+ Under "Year" choose your range (most recently I've selected the full available range of 2001 - recent **Watch out here - keep old year if range changes and add onto new download**)  
+ Select the value "GDP".  

When you've downloaded this csv, open it and replace the title row with the year (as all of the values are GDPs), delete the header line and data notes at the bottom, rename region to "NAME", and select all of the numerical data and make them "General" format. You can also quickly delimit the NAMEs to remove the ", Tennessee" off of the end of the counties and change "USA" to "US". This can also be done here, of course. Save as 'JobsEQ_GDPAnnual_GeosExceptPlaces.csv'

In [29]:
#GDP import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_GDPAnnual_GeosExceptPlaces.csv')
data.head(2)

,NAME,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,US,1.058190e+13,1.092910e+13,1.145640e+13,1.221720e+13,1.303920e+13,1.381560e+13,1.447420e+13,1.476980e+13,1.447810e+13,1.504900e+13,1.559970e+13,1.625400e+13,1.684320e+13,1.755070e+13,1.820570e+13,1.869490e+13,1.947960e+13,2.052680e+13,2.137250e+13,2.089370e+13,2.299610e+13
1,Tennessee,1.864350e+11,1.957600e+11,2.051200e+11,2.193220e+11,2.316270e+11,2.435780e+11,2.486910e+11,2.542420e+11,2.523330e+11,2.577760e+11,2.698140e+11,2.863420e+11,2.958630e+11,3.061530e+11,3.252940e+11,3.364140e+11,3.498380e+11,3.613800e+11,3.769150e+11,3.695720e+11,4.182940e+11


In [30]:
data['NAME'].unique()

array(['US', 'Tennessee', 'GNRC', 'MPO', 'GNRC Region', 'Cheatham County',
       'Davidson County', 'Dickson County', 'Houston County',
       'Humphreys County', 'Maury County', 'Montgomery County',
       'Robertson County', 'Rutherford County', 'Stewart County',
       'Sumner County', 'Trousdale County', 'Williamson County',
       'Wilson County'], dtype=object)

In [31]:
#make a list of the columns to melt from wide to long format with the year as a variable and naming the value "GDP"
cols = data.columns
data = data.melt(var_name = 'Year', id_vars = ['NAME'], value_vars = cols, value_name = 'GDP:GDP')

In [32]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [33]:
data.head()

,NAME,Year,GDP:GDP
0,United States,2001,10581900000000.0
1,Tennessee,2001,186435000000.0
2,GNRC,2001,55545232000.0
3,MPO,2001,53769753000.0
4,GNRC Region,2001,58130905000.0


In [34]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [35]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

GDP:GDP                                                                                                    \
Year                                2001          2002          2003          2004          2005          2006          2007          2008   
NAME                                                                                                                                         
Cheatham County, Tennessee  4.590110e+08  5.296220e+08  5.643490e+08  6.247250e+08  6.785940e+08  7.196860e+08  7.098450e+08  7.842820e+08   
Davidson County, Tennessee  3.403081e+10  3.512616e+10  3.727170e+10  3.900497e+10  4.139817e+10  4.386193e+10  4.472193e+10  4.653565e+10   

                                                                                                                                            \
Year                                2009          2010          2011          2012          2013          2014          2015          2016   
NAME                                                                                                                                         
Cheatham County, Tennessee  7.557930e+08  7.694610e+08  7.600230e+08  7.668240e+08  8.288380e+08  8.198790e+08  9.209110e+08  9.394920e+08   
Davidson County, Tennessee  4.719965e+10  4.855240e+10  5.124097e+10  5.435087e+10  5.613963e+10  5.925130e+10  6.270032e+10  6.553752e+10   

                                                                                                  
Year                                2017          2018          2019          2020          2021  
NAME                                                                                              
Cheatham County, Tennessee  1.026863e+09  1.071879e+09  1.104530e+09  1.094723e+09  1.233382e+09  
Davidson County, Tennessee  6.892946e+10  7.293132e+10  7.662392e+10  7.174367e+10  8.255004e+10

In [36]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

GDP:GDP                                                                                                    \
                                    2001          2002          2003          2004          2005          2006          2007          2008   
                                    None          None          None          None          None          None          None          None   
NAME                                                                                                                                         
Cheatham County, Tennessee  4.590110e+08  5.296220e+08  5.643490e+08  6.247250e+08  6.785940e+08  7.196860e+08  7.098450e+08  7.842820e+08   
Davidson County, Tennessee  3.403081e+10  3.512616e+10  3.727170e+10  3.900497e+10  4.139817e+10  4.386193e+10  4.472193e+10  4.653565e+10   
Dickson County, Tennessee   8.536360e+08  8.364840e+08  9.001090e+08  9.450570e+08  1.030411e+09  1.080384e+09  1.092047e+09  1.042446e+09   

                                                                                                                                            \
                                    2009          2010          2011          2012          2013          2014          2015          2016   
                                    None          None          None          None          None          None          None          None   
NAME                                                                                                                                         
Cheatham County, Tennessee  7.557930e+08  7.694610e+08  7.600230e+08  7.668240e+08  8.288380e+08  8.198790e+08  9.209110e+08  9.394920e+08   
Davidson County, Tennessee  4.719965e+10  4.855240e+10  5.124097e+10  5.435087e+10  5.613963e+10  5.925130e+10  6.270032e+10  6.553752e+10   
Dickson County, Tennessee   1.002546e+09  1.019388e+09  1.098554e+09  1.159218e+09  1.291117e+09  1.350150e+09  1.449488e+09  1.532418e+09   

                                                                                                  
                                    2017          2018          2019          2020          2021  
                                    None          None          None          None          None  
NAME                                                                                              
Cheatham County, Tennessee  1.026863e+09  1.071879e+09  1.104530e+09  1.094723e+09  1.233382e+09  
Davidson County, Tennessee  6.892946e+10  7.293132e+10  7.662392e+10  7.174367e+10  8.255004e+10  
Dickson County, Tennessee   1.616509e+09  1.638125e+09  1.750963e+09  1.713265e+09  1.880187e+09

In [37]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
first_level

['GDP:GDP']

In [38]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [39]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [40]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ BEA'

In [41]:
#final check
data.head()

,NAME,Year,Time Frame,GDP:GDP,GDP:GDP % Change,GDP:GDP Change,GEO_ID,Source
0,"Cheatham County, Tennessee",2001,None,459011000.0,NaN,NaN,0500000US47021,JobsEQ BEA
1,"Cheatham County, Tennessee",2002,None,529622000.0,NaN,NaN,0500000US47021,JobsEQ BEA
2,"Cheatham County, Tennessee",2003,None,564349000.0,NaN,NaN,0500000US47021,JobsEQ BEA
3,"Cheatham County, Tennessee",2004,None,624725000.0,NaN,NaN,0500000US47021,JobsEQ BEA
4,"Cheatham County, Tennessee",2005,None,678594000.0,NaN,NaN,0500000US47021,JobsEQ BEA


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4389 entries, 0 to 4388
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NAME              4389 non-null   object 
 1   Year              4389 non-null   object 
 2   Time Frame        4389 non-null   object 
 3   GDP:GDP           399 non-null    float64
 4   GDP:GDP % Change  3990 non-null   float64
 5   GDP:GDP Change    3990 non-null   float64
 6   GEO_ID            3465 non-null   object 
 7   Source            4389 non-null   object 
dtypes: float64(3), object(5)
memory usage: 274.4+ KB


In [43]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_GDP_Annual_Change', conn, if_exists = 'replace', index = False)

4389

Awards:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Education" dataset  
+ Select the following dimensions: Region and Schools, Year, and Award Level   
+ Under "Region" select "GNRC Region - All Schools", and then sub counties    
+ Under "Year" choose your range (most recently I've selected the full available range of 2010 - recent **Watch out here - keep old year if range changes and add onto new download**)  
+ Under Award Level leave "All Award Levels", and then all subs  
+ Select the value "Awards"  
+ Swap rows & columns so the "Regions & Schools" data is at the top, "Award Level" is on the side, as well as "Year"  

When you've downloaded this csv, open it and replace the title row with the geography (as all of the values are all awards) while keeping the headers for award level and year, then delete the top line and data notes at the bottom, and select all of the numerical data and make them "General" format. Save as 'JobsEQ_AwardsAnnual_GeosExceptPlaces.csv'

In [70]:
#Awards import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_AwardsAnnual_GeosExceptPlaces.csv')
data.head(2)

,Award Level,Year,GNRC Region-All Schools,"Cheatham County, Tennessee-All Schools (47021)","Davidson County, Tennessee-All Schools (47037)","Dickson County, Tennessee-All Schools (47043)","Houston County, Tennessee-All Schools (47083)","Humphreys County, Tennessee-All Schools (47085)","Maury County, Tennessee-All Schools (47119)","Montgomery County, Tennessee-All Schools (47125)","Robertson County, Tennessee-All Schools (47147)","Rutherford County, Tennessee-All Schools (47149)","Stewart County, Tennessee-All Schools (47161)","Sumner County, Tennessee-All Schools (47165)","Trousdale County, Tennessee-All Schools (47169)","Williamson County, Tennessee-All Schools (47187)","Wilson County, Tennessee-All Schools (47189)",Unnamed: 17
0,All Award Levels,2010,25779,NaN,15864,294.0,NaN,NaN,699.0,2166.0,NaN,4891,NaN,1033.0,242.0,55.0,535.0,NaN
1,All Award Levels,2011,28038,NaN,17182,394.0,NaN,NaN,676.0,2517.0,NaN,5338,NaN,1033.0,249.0,96.0,553.0,NaN


In [71]:
#don't know where this came from, can revisit later... but drop weird column
data.drop(columns = 'Unnamed: 17', inplace = True)

In [72]:
#make a list of columns to take from wide to long format naming the column of current columns "NAME" and the value "Awards"
cols = data.columns
data = data.melt(var_name = 'NAME', id_vars = ['Award Level', 'Year'], value_vars = cols, value_name = 'Awards')
data.head(2)

,Award Level,Year,NAME,Awards
0,All Award Levels,2010,GNRC Region-All Schools,25779.0
1,All Award Levels,2011,GNRC Region-All Schools,28038.0


In [73]:
#split the NAME on common delimiters and expand the split into a new dataframe, where you'll take the first instance and write-over the original "NAME" and strip
boop = data['NAME'].str.split(r"\-|,", expand = True)
data['NAME'] = boop[0].str.strip()

In [74]:
#make lists of years and geographies to loop through and group the different award levels
years = list(data['Year'].unique())
geos = list(data['NAME'].unique())

In [75]:
#format the column headers so that each individual instance can be looped through, and you can convert to float types before by dropping null then converting
data = data.set_index(['Award Level', 'Year', 'NAME']).transpose()
data.head()

Award Level All Award Levels                                                                                                              \
Year                    2010        2011        2012        2013        2014        2015        2016        2017        2018        2019   
NAME             GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards               25779.0     28038.0     30017.0     28404.0     28353.0     28549.0     28580.0     28689.0     27726.0     27785.0   

Award Level                         Certificate < 1 Yr                                                                                      \
Year               2020        2021               2010        2011        2012        2013        2014        2015        2016        2017   
NAME        GNRC Region GNRC Region        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards          28519.0     29332.0             2404.0      2065.0      2891.0      2070.0      2192.0      2370.0      2357.0      2479.0   

Award Level                                                 Certificate 1+ but < 2 Yr                                                              \
Year               2018        2019        2020        2021                      2010        2011        2012        2013        2014        2015   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region               GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards           1644.0      1949.0      2048.0      2251.0                    2551.0      4466.0      5179.0      4114.0      3697.0      3549.0   

Award Level                                                                         Associate's                                                  \
Year               2016        2017        2018        2019        2020        2021        2010        2011        2012        2013        2014   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards           3418.0      3304.0      3107.0      2738.0      1824.0      2063.0      3416.0      4153.0      4459.0      4008.0      3644.0   

Award Level                                                                                     Certificate 2+ but < 4 Yr                          \
Year               2015        2016        2017        2018        2019        2020        2021                      2010        2011        2012   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region               GNRC Region GNRC Region GNRC Region   
Awards           3818.0      3749.0      3766.0      4010.0      4160.0      4849.0      5212.0                    1293.0       644.0       177.0   

Award Level                                                                                                              Bachelor's              \
Year               2013        2014        2015        2016        2017        2018        2019        2020        2021        2010        2011   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards            118.0       254.0       198.0       253.0       186.0       193.0       234.0       640.0       634.0     10772.0     11215.0   

Award Level                                                                                                                          \
Year               2012        2013        2014        2015        2016        2017        2018        2019        2020        2021   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards          11364.0     11904.0     12237.0     12252.0     12501.0     12637.0     12565

In [76]:
#fill nulls with 0.0 and convert to float
data = data.fillna(0.0)
cols = data.columns
data[cols] = data[cols].astype(float)

In [77]:
#loop through all columns grouping certain certificates and higher levels of awards for each geography and year then transpose & reset idnex without dropping
for year in years:
    for geo in geos:
        data[('Certificate < 2 Years', year, '{}'.format(geo))] = data[('Certificate < 1 Yr', year, '{}'.format(geo))] + data[('Certificate 1+ but < 2 Yr',
                                                                                                                                  year, '{}'.format(geo))]
        data[('Certificate > 2 Years', year, '{}'.format(geo))] = data[('Certificate 2+ but < 4 Yr', year, '{}'.format(geo))]
        thelist = [data[("Master's", year, '{}'.format(geo))], data[("Post-Master's", year, '{}'.format(geo))], 
                   data[("Doctorate", year, '{}'.format(geo))]]
        data[("Master's or Above", year, '{}'.format(geo))] = sum(thelist)
        thelist = [data[("Bachelor's", year, '{}'.format(geo))], data[("Post-Baccalaureate", year, '{}'.format(geo))]]
        data[("Bachelor's", year, '{}'.format(geo))] = sum(thelist)
        data = data.drop(columns = [('Certificate < 1 Yr', year, '{}'.format(geo)), ('Certificate 1+ but < 2 Yr', year, '{}'.format(geo)), 
                                      ('Certificate 2+ but < 4 Yr', year, '{}'.format(geo)), ('Post-Baccalaureate', year, '{}'.format(geo))])

In [78]:
#transpose the dataframe and reset the index without dropping
data = data.transpose().reset_index(drop = False)

In [79]:
data.head()

,Award Level,Year,NAME,Awards
0,All Award Levels,2010,GNRC Region,25779.0
1,All Award Levels,2011,GNRC Region,28038.0
2,All Award Levels,2012,GNRC Region,30017.0
3,All Award Levels,2013,GNRC Region,28404.0
4,All Award Levels,2014,GNRC Region,28353.0


In [80]:
#pivot to take from long to wide format on award levels
data = data.pivot(index = ['Year', 'NAME'], columns = 'Award Level')['Awards']

In [81]:
data.head()

Award Level           All Award Levels  Associate's  Bachelor's  Certificate < 2 Years  Certificate > 2 Years  Doctorate  Master's  \
Year NAME                                                                                                                            
2010 Cheatham County               0.0          0.0         0.0                    0.0                    0.0        0.0       0.0   
     Davidson County           15864.0       1711.0      5742.0                 3221.0                 1164.0      958.0    3045.0   
     Dickson County              294.0          0.0         0.0                  248.0                   46.0        0.0       0.0   
     GNRC Region               25779.0       3416.0     10823.0                 4955.0                 1293.0      978.0    4151.0   
     Houston County                0.0          0.0         0.0                    0.0                    0.0        0.0       0.0   

Award Level           Master's or Above  Post-Master's  
Year NAME                                               
2010 Cheatham County                0.0            0.0  
     Davidson County             4026.0           23.0  
     Dickson County                 0.0            0.0  
     GNRC Region                 5292.0          163.0  
     Houston County                 0.0            0.0

In [82]:
#reset and remove name from index
data = data.reset_index(drop = False).rename_axis(None, axis=1).reset_index(drop = True)

In [83]:
#set the index as year and name to remove them from the set of columns for prefix, add the prefix, then drop and keep the indices
data.set_index(['Year', 'NAME'], inplace = True)
data = data.add_prefix('Awards:').reset_index(drop = False)

In [84]:
#calculate percentages
data["Awards %:Associate's"] = percent(data["Awards:Associate's"], data['Awards:All Award Levels'])
data["Awards %:Bachelor's"] = percent(data["Awards:Bachelor's"], data['Awards:All Award Levels'])
data["Awards %:Certificate < 2 Years"] = percent(data["Awards:Certificate < 2 Years"], data['Awards:All Award Levels'])
data["Awards %:Certificate > 2 Years"] = percent(data["Awards:Certificate > 2 Years"], data['Awards:All Award Levels'])
data["Awards %:Doctorate"] = percent(data["Awards:Doctorate"], data['Awards:All Award Levels'])
data["Awards %:Master's"] = percent(data["Awards:Master's"], data['Awards:All Award Levels'])
data["Awards %:Master's or Above"] = percent(data["Awards:Master's or Above"], data['Awards:All Award Levels'])
data["Awards %:Post-Master's"] = percent(data["Awards:Post-Master's"], data['Awards:All Award Levels'])

In [85]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [86]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [87]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Awards %:Associate's                                                                                              \
Year                                       2010       2011      2012       2013      2014      2015      2016      2017      2018      2019   
NAME                                                                                                                                          
Cheatham County, Tennessee                  NaN        NaN       NaN        NaN  0.000000  0.000000  0.000000       NaN       NaN       NaN   
Davidson County, Tennessee            10.785426  12.903038  13.32947  11.077863  9.585555  9.653643  9.143992  8.610736  9.004679  9.094468   

                                               Awards %:Bachelor's                                                                             \
Year                            2020      2021                2010      2011       2012       2013       2014       2015      2016       2017   
NAME                                                                                                                                            
Cheatham County, Tennessee       NaN       NaN                 NaN       NaN        NaN        NaN   0.000000   0.000000   0.00000        NaN   
Davidson County, Tennessee  8.499967  8.931766           36.195159  34.27424  33.387424  36.635115  38.075618  38.704458  39.86877  40.093355   

                                                                       Awards %:Certificate < 2 Years                                               \
Year                             2018       2019       2020       2021                           2010       2011       2012       2013        2014   
NAME                                                                                                                                                 
Cheatham County, Tennessee        NaN        NaN        NaN        NaN                            NaN        NaN        NaN        NaN  100.000000   
Davidson County, Tennessee  42.267513  42.660491  43.588728  42.793253                      20.303833  27.034105  26.850188  23.847328   22.424867   

                                                                                                          Awards %:Certificate > 2 Years            \
Year                              2015        2016       2017       2018       2019       2020       2021                           2010      2011   
NAME                                                                                                                                                 
Cheatham County, Tennessee  100.000000  100.000000        NaN        NaN        NaN        NaN        NaN                            NaN       NaN   
Davidson County, Tennessee   22.183605   21.093186  21.594399  18.086265  16.492693  11.746849  12.854587                       7.337368  2.322198   

                                                                                                                             Awards %:Doctorate  \
Year                            2012      2013     2014      2015     2016      2017      2018      2019      2020      2021               2010   
NAME                                                                                                                                              
Cheatham County, Tennessee       NaN       NaN  0.00000  0.000000  0.00000       NaN       NaN       NaN       NaN       NaN                NaN   
Davidson County, Tennessee  0.365112  0.244275  0.27872  0.269655  0.70431  0.319371  0.378132  0.685021  3.339273  3.002811            6.03883   

                                                                                                                                            \
Year                            2011      2012      2013      2014      2015      2016      2017      2018      2019       2020       2021   
NAME                                                           

In [88]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(2)

Awards %:Associate's                                                                                              \
                                           2010       2011      2012       2013      2014      2015      2016      2017      2018      2019   
                                           None       None      None       None      None      None      None      None      None      None   
NAME                                                                                                                                          
Cheatham County, Tennessee                  NaN        NaN       NaN        NaN  0.000000  0.000000  0.000000       NaN       NaN       NaN   
Davidson County, Tennessee            10.785426  12.903038  13.32947  11.077863  9.585555  9.653643  9.143992  8.610736  9.004679  9.094468   

                                               Awards %:Bachelor's                                                                             \
                                2020      2021                2010      2011       2012       2013       2014       2015      2016       2017   
                                None      None                None      None       None       None       None       None      None       None   
NAME                                                                                                                                            
Cheatham County, Tennessee       NaN       NaN                 NaN       NaN        NaN        NaN   0.000000   0.000000   0.00000        NaN   
Davidson County, Tennessee  8.499967  8.931766           36.195159  34.27424  33.387424  36.635115  38.075618  38.704458  39.86877  40.093355   

                                                                       Awards %:Certificate < 2 Years                                               \
                                 2018       2019       2020       2021                           2010       2011       2012       2013        2014   
                                 None       None       None       None                           None       None       None       None        None   
NAME                                                                                                                                                 
Cheatham County, Tennessee        NaN        NaN        NaN        NaN                            NaN        NaN        NaN        NaN  100.000000   
Davidson County, Tennessee  42.267513  42.660491  43.588728  42.793253                      20.303833  27.034105  26.850188  23.847328   22.424867   

                                                                                                          Awards %:Certificate > 2 Years            \
                                  2015        2016       2017       2018       2019       2020       2021                           2010      2011   
                                  None        None       None       None       None       None       None                           None      None   
NAME                                                                                                                                                 
Cheatham County, Tennessee  100.000000  100.000000        NaN        NaN        NaN        NaN        NaN                            NaN       NaN   
Davidson County, Tennessee   22.183605   21.093186  21.594399  18.086265  16.492693  11.746849  12.854587                       7.337368  2.322198   

                                                                                                                             Awards %:Doctorate  \
                                2012      2013     2014      2015     2016      2017      2018      2019      2020      2021               2010   
                                None      None     None      None     None      None      None      None      None      None               None   
NAME                                                  

In [89]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [90]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [91]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [92]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ NCES'

In [93]:
#final check
data.head(3)

,NAME,Year,Time Frame,Awards %:Associate's,Awards %:Bachelor's,Awards %:Certificate < 2 Years,Awards %:Certificate > 2 Years,Awards %:Doctorate,Awards %:Master's,Awards %:Master's or Above,Awards %:Post-Master's,Awards:All Award Levels,Awards:All Award Levels % Change,Awards:All Award Levels Change,Awards:Associate's,Awards:Associate's % Change,Awards:Associate's Change,Awards:Bachelor's,Awards:Bachelor's % Change,Awards:Bachelor's Change,Awards:Certificate < 2 Years,Awards:Certificate < 2 Years % Change,Awards:Certificate < 2 Years Change,Awards:Certificate > 2 Years,Awards:Certificate > 2 Years % Change,Awards:Certificate > 2 Years Change,Awards:Doctorate,Awards:Doctorate % Change,Awards:Doctorate Change,Awards:Master's,Awards:Master's % Change,Awards:Master's Change,Awards:Master's or Above,Awards:Master's or Above % Change,Awards:Master's or Above Change,Awards:Post-Master's,Awards:Post-Master's % Change,Awards:Post-Master's Change,GEO_ID,Source
0,"Cheatham County, Tennessee",2010,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0500000US47021,JobsEQ NCES
1,"Cheatham County, Tennessee",2011,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0500000US47021,JobsEQ NCES
2,"Cheatham County, Tennessee",2012,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0500000US47021,JobsEQ NCES


In [94]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   NAME                                   1170 non-null   object 
 1   Year                                   1170 non-null   object 
 2   Time Frame                             1170 non-null   object 
 3   Awards %:Associate's                   123 non-null    float64
 4   Awards %:Bachelor's                    123 non-null    float64
 5   Awards %:Certificate < 2 Years         123 non-null    float64
 6   Awards %:Certificate > 2 Years         123 non-null    float64
 7   Awards %:Doctorate                     123 non-null    float64
 8   Awards %:Master's                      123 non-null    float64
 9   Awards %:Master's or Above             123 non-null    float64
 10  Awards %:Post-Master's                 123 non-null    float64
 11  Awar

In [95]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_Awards_Annual_Change', conn, if_exists = 'replace', index = False)

1170

This retrieves historical and current annual occupational employment disaggregated by high level occupation group.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ Select the "Occupation" dataset, year/quarter is the only option, we'll have to make it into an annual number  
+ Select parameters: "Region", "Occupation", "Year/Quarter"  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Places"  
+ For "Year/Quarter" select as many as possible  
+ Under "Occupation" expand "Total-All Occupations" and select the combination of 2 digit SOCS codes and built groups in Jobs EQ as described below, leaving Total as well  

Occupations to Select:  
(SOC Codes)  
+ Total - All Occupations  
+ Healthcare Practitioners and Technical Occupations (29)  
+ Healthcare Support Occupations (31)  
+ Protective Service Occupations (33)  
+ Production Occupations (51)  
+ Transportation and Material Moving Occupations (53)  
(Custom Created Groups)  
+ Business & Financial (11: Management & 13: Business & Financial Operations)  
+ Computer, Mathematical, & Engineering (15: Computer & Mathematical, 17: Architecture & Engineering, & 19: Life, Physical, & Social Science)  
+ Education, Legal, Social, Arts, & Media (21: Community & Social Service, 23: Legal, 25: Educational Instruction & Library, & 27: Art, Design, Entertainment, Sports, & Media)  
+ Food, Property, & Personal Services (35: Food Preparation & Serving, 37: Building & Grounds Cleaning & Maintenance, & 39: Personal Care & Service)  
+ Sales & Office Support (41: Sales & Related & 43: Office & Administrative Support)  
+ Natural Resources, Construction, & Maintenance (45: Farming Fishing & Forestry, 47: Construction & Extraction, & 49: Installation Maintenance & Repair)

+ Make sure Occupation Data is on the top, and Year/Quarter and Region are on the side (this will make the small bit of Excel formatting easier)  
+ Select value: Employment, place of work  

This may exceed the drafting size, in which case you'll have to queue for download. Once it is completed, open and delete the top row that says "Occupation Data", and delete the third row that repeats "Employment (Place of Work)". Rename the FIPS column to GEO_ID, and "Region" to "NAME". Use text to columns to change the GEO_ID column to text, and select all numerical data and format to "general". Save in the "Data Downloads" folder as "JobsEQ_OccupationalEmploymentAnnual_AllGeos.csv"

In [29]:
#import and examine, dtype str so the GEO_ID formatting doesn't get messed up and so that we can split the time column
data = pd.read_csv('../../Data Downloads/JobsEQ_OccupationalEmploymentAnnual_AllGeos.csv', dtype = str)
data = data.drop(columns = 'GEO_ID')
data.head(2)

,Time,NAME,Total - All Occupations,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Support Occupations (31-0000),Protective Service Occupations (33-0000),Production Occupations (51-0000),Transportation and Material Moving Occupations (53-0000),Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,2012Q3,USA,142247745,7446442.208,5438164.315,3187440.814,9189061.611,11168810.96,14832563.75,6535636.51,14906649.68,20682569.59,35932119.13,12928286.45
1,2012Q3,Tennessee,2906688.129,171922.1486,107353.8776,63180.76471,246548.9179,290714.4475,276518.5131,99894.06596,271615.8749,406234.0525,719088.9103,253616.5561


In [30]:
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)
data.head(2)

,NAME,Time,Total - All Occupations,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Support Occupations (31-0000),Protective Service Occupations (33-0000),Production Occupations (51-0000),Transportation and Material Moving Occupations (53-0000),Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,United States,2012Q3,142247745,7446442.208,5438164.315,3187440.814,9189061.611,11168810.96,14832563.75,6535636.51,14906649.68,20682569.59,35932119.13,12928286.45
1,Tennessee,2012Q3,2906688.129,171922.1486,107353.8776,63180.76471,246548.9179,290714.4475,276518.5131,99894.06596,271615.8749,406234.0525,719088.9103,253616.5561
2,MPO,2012Q3,815188.578,49643.31623,30407.00286,19495.41212,54634.99673,69995.10703,82974.5814,29997.06868,84246.65723,113103.4739,215730.2953,64960.66649


In [32]:
#use str.split to break out the Time column and create a year and quarter column
boop = data['Time'].str.split(pat = 'Q', expand = True)
data['Year'] = boop[0]
data['Quarter'] = boop[1]
data.drop(columns = 'Time', inplace = True)

In [18]:
#we want the last full annual number, which is 2021 at this time, so knock out the handful of quarters in 2012 and 2022
data = data.loc[(data['Year'] != '2012')&(data['Year'] != '2022')]

In [19]:
#to average out years, make a list of years and of geographies in the dataframe
years = list(data.Year.unique())
geos = list(data.NAME.unique())

In [20]:
#set the index to be the combination of quarter year and name, as this gives unique instances to loop through, and transpose
data = data.set_index(['Quarter', 'Year', 'NAME']).transpose()

In [21]:
#make sure the other columns are float datatypes to perform aggregations
cols = list(data.columns)
data[cols] = data[cols].astype(float)

In [22]:
#loop through all four quarters, the list of unique years, and unique geographies to take the average of all four quarters
cols = data.columns
for year in years:
    for geo in geos:
        thelist = [data[('1', year, '{}'.format(geo))], data[('2', year, '{}'.format(geo))], data[('3', year, '{}'.format(geo))], 
                  data[('4', year, '{}'.format(geo))]]
        data[('All', year, '{}'.format(geo))] = (sum(thelist))/4
data = data.drop(columns = cols)

In [23]:
#transpose the data and reset the index without dropping, drop the quarter column
data = data.transpose().reset_index(drop = False).drop(columns = 'Quarter')

In [24]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Healthcare Practitioners and Technical Occupations (29-0000)'], data['Healthcare Support Occupations (31-0000)'], 
           data['Protective Service Occupations (33-0000)'], data['Production Occupations (51-0000)'], 
           data['Transportation and Material Moving Occupations (53-0000)'], data['Business & Financial'],
           data['Computer, Mathematical, & Engineering'], data['Education, Legal, Social, Arts, & Media'], 
           data['Food, Property, & Personal Services'], data['Sales & Office Support'], data['Natural Resources, Construction, & Maintenance']]
data['Total - All Occupations'] = sum(thelist)

In [25]:
#rename columns to account for the disparity between custom groups and SOC 2 digit formatting
data = data.rename(columns = {'Healthcare Practitioners and Technical Occupations (29-0000)': 'Healthcare Practitioners & Technical',
                              'Healthcare Support Occupations (31-0000)': 'Healthcare Support', 
                              'Protective Service Occupations (33-0000)': 'Protective Services',
                              'Production Occupations (51-0000)': 'Production', 
                              'Transportation and Material Moving Occupations (53-0000)': 'Transportation & Material Moving', 
                             'Total - All Occupations': 'Total'})

In [26]:
years = list(data['Year'].unique())

In [27]:
#set the index to year and name so that we can add a prefix to all of the other columns, before resetting the index without dropping
data = data.set_index(['Year', 'NAME'])
data = data.add_prefix('Occ:')#.reset_index(drop = False)
data = data.transpose()

In [28]:
cols = data.columns
data[cols] = data[cols].astype(float)

In [43]:
for time in years:
        RuthInc = [data[('{}'.format(time), 'Eagleville city, Tennessee')],data[('{}'.format(time), 'La Vergne city, Tennessee')],
               data[('{}'.format(time), 'Murfreesboro city, Tennessee')],data[('{}'.format(time), 'Smyrna town, Tennessee')]]
        data[('{}'.format(time), 'Rutherford Incorporated')] = sum(RuthInc)
        data[('{}'.format(time), 'Rutherford Unincorporated')] = data[('{}'.format(time), 'Rutherford County, Tennessee')] - data[('{}'.format(time), 'Rutherford Incorporated')]
        WilsonInc = [data[('{}'.format(time), 'Lebanon city, Tennessee')],data[('{}'.format(time), 'Mount Juliet city, Tennessee')],
                     data[('{}'.format(time), 'Watertown city, Tennessee')]]
        data[('{}'.format(time), 'Wilson Incorporated')] = sum(WilsonInc)
        data[('{}'.format(time), 'Wilson Unincorporated')] = data[('{}'.format(time), 'Wilson County, Tennessee')] - data[('{}'.format(time), 'Wilson Incorporated')]
        CheathInc = [data[('{}'.format(time), 'Ashland City town, Tennessee')],data[('{}'.format(time), 'Kingston Springs town, Tennessee')],
                     data[('{}'.format(time), 'Pegram town, Tennessee')],data[('{}'.format(time), 'Pleasant View city, Tennessee')]]
        data[('{}'.format(time), 'Cheatham Incorporated')] = sum(CheathInc)
        data[('{}'.format(time), 'Cheatham Unincorporated')] = data[('{}'.format(time), 'Cheatham County, Tennessee')] - data[('{}'.format(time), 'Cheatham Incorporated')]
        DicksInc = [data[('{}'.format(time), 'Burns town, Tennessee')],data[('{}'.format(time), 'Charlotte town, Tennessee')],
                    data[('{}'.format(time), 'Dickson city, Tennessee')],data[('{}'.format(time), 'Slayden town, Tennessee')],
                    data[('{}'.format(time), 'Vanleer town, Tennessee')],data[('{}'.format(time), 'White Bluff town, Tennessee')]]
        data[('{}'.format(time), 'Dickson Incorporated')] = sum(DicksInc)
        data[('{}'.format(time), 'Dickson Unincorporated')] = data[('{}'.format(time), 'Dickson County, Tennessee')] - data[('{}'.format(time), 'Dickson Incorporated')]
        HumphInc = [data[('{}'.format(time), 'McEwen city, Tennessee')],data[('{}'.format(time), 'New Johnsonville city, Tennessee')],
                    data[('{}'.format(time), 'Waverly city, Tennessee')]]
        data[('{}'.format(time), 'Humphreys Incorporated')] = sum(HumphInc)
        data[('{}'.format(time), 'Humphreys Unincorporated')] = data[('{}'.format(time), 'Humphreys County, Tennessee')] - data[('{}'.format(time), 'Humphreys Incorporated')]
        data[('{}'.format(time), 'Montgomery Incorporated')] = data[('{}'.format(time), 'Clarksville city, Tennessee')]
        data[('{}'.format(time), 'Montgomery Unincorporated')] = data[('{}'.format(time), 'Montgomery County, Tennessee')] - data[('{}'.format(time), 'Montgomery Incorporated')]

In [44]:
data = data.transpose()
data.head()

Occ:Total  Occ:Healthcare Practitioners & Technical  Occ:Healthcare Support  Occ:Protective Services  Occ:Production  \
Year NAME                                                                                                                                     
2013 United States  1.442553e+08                              7.602078e+06            5.792251e+06             3.211846e+06    9.241163e+06   
     Tennessee      2.942359e+06                              1.754508e+05            1.092862e+05             6.387145e+04    2.506506e+05   
     MPO            8.388774e+05                              5.197844e+04            3.086795e+04             1.946964e+04    5.798775e+04   
     GNRC           8.978335e+05                              5.412080e+04            3.256645e+04             2.051455e+04    6.341400e+04   
     GNRC Region    9.296939e+05                              5.600879e+04            3.399026e+04             2.103600e+04    6.650445e+04   

                    Occ:Transportation & Material Moving  Occ:Business & Financial  Occ:Computer, Mathematical, & Engineering  \
Year NAME                                                                                                                       
2013 United States                          1.137122e+07              1.533409e+07                               6.758342e+06   
     Tennessee                              2.963182e+05              2.840729e+05                               1.034084e+05   
     MPO                                    7.295762e+04              8.665146e+04                               3.130359e+04   
     GNRC                                   7.803496e+04              9.170417e+04                               3.306256e+04   
     GNRC Region                            8.056577e+04              9.493133e+04                               3.388182e+04   

                    Occ:Education, Legal, Social, Arts, & Media  Occ:Food, Property, & Personal Services  Occ:Sales & Office Support  \
Year NAME                                                                                                                              
2013 United States                                 1.497493e+07                             2.082948e+07                3.603863e+07   
     Tennessee                                     2.712434e+05                             4.139311e+05                7.215578e+05   
     MPO                                           8.435870e+04                             1.173205e+05                2.199417e+05   
     GNRC                                          9.143700e+04                             1.277296e+05                2.329258e+05   
     GNRC Region                                   9.465788e+04                             1.324553e+05                2.406541e+05   

                    Occ:Natural Resources, Construction, & Maintenance  
Year NAME                                                               
2013 United States                                       1.310124e+07   
     Tennessee                                           2.525685e+05   
     MPO                                                 6.604006e+04   
     GNRC                                                7.232366e+04   
     GNRC Region                                         7.500829e+04

In [45]:
data = data.reset_index(drop = False)

In [46]:
#melt this data down using the columns name and year to distinguish specific instances, and then attributing total employment per occupation
#name it separately because it bugs out if you call this data and merge later
cols = data.columns
thing = data.melt(var_name = 'Occupation', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Employment')
thing.head(2)

,NAME,Year,Occupation,Employment
0,United States,2013,Occ:Total,1.442553e+08
1,Tennessee,2013,Occ:Total,2.942359e+06


In [47]:
#create a separate dataframe where we will calculate percent of the total for each occupation group
percents = data

In [48]:
#calculate percent of total
percents['Occ:Healthcare Practitioners & Technical'] = percent(percents['Occ:Healthcare Practitioners & Technical'], percents['Occ:Total'])
percents['Occ:Healthcare Support'] = percent(percents['Occ:Healthcare Support'], percents['Occ:Total'])
percents['Occ:Protective Services'] = percent(percents['Occ:Protective Services'], percents['Occ:Total'])
percents['Occ:Production'] = percent(percents['Occ:Production'], percents['Occ:Total'])
percents['Occ:Transportation & Material Moving'] = percent(percents['Occ:Transportation & Material Moving'], percents['Occ:Total'])
percents['Occ:Business & Financial'] = percent(percents['Occ:Business & Financial'], percents['Occ:Total'])
percents['Occ:Computer, Mathematical, & Engineering'] = percent(percents['Occ:Computer, Mathematical, & Engineering'], percents['Occ:Total'])
percents['Occ:Education, Legal, Social, Arts, & Media'] = percent(percents['Occ:Education, Legal, Social, Arts, & Media'], percents['Occ:Total'])
percents['Occ:Food, Property, & Personal Services'] = percent(percents['Occ:Food, Property, & Personal Services'], percents['Occ:Total'])
percents['Occ:Sales & Office Support'] = percent(percents['Occ:Sales & Office Support'], percents['Occ:Total'])
percents['Occ:Natural Resources, Construction, & Maintenance'] = percent(percents['Occ:Natural Resources, Construction, & Maintenance'], percents['Occ:Total'])

In [49]:
#similarly melt this dataframe
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Occupation', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Percent of Employment')
percents.head()

,NAME,Year,Occupation,Percent of Employment
0,United States,2013,Occ:Total,1.442553e+08
1,Tennessee,2013,Occ:Total,2.942359e+06
2,MPO,2013,Occ:Total,8.388774e+05
3,GNRC,2013,Occ:Total,8.978335e+05
4,GNRC Region,2013,Occ:Total,9.296939e+05


In [50]:
#we never renamed the total column when we duplicated the dataframe for percents, we want this to be null so just transpose, drop it, and transpose back
percents = percents.set_index('Occupation').transpose()
percents = percents.drop(columns = 'Occ:Total')
percents = percents.transpose().reset_index(drop = False)

In [51]:
#merge total employment and percent of total employment
data = thing.merge(percents, on = ['Occupation', 'Year', 'NAME'], how = 'outer')
data.head(3)

,NAME,Year,Occupation,Employment,Percent of Employment
0,United States,2013,Occ:Total,1.442553e+08,NaN
1,Tennessee,2013,Occ:Total,2.942359e+06,NaN
2,MPO,2013,Occ:Total,8.388774e+05,NaN


In [52]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos before resetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus).reset_index(drop = False)

In [53]:
#set the index to "index" (the variable names) and transpose, then reset the index without dropping it
data = data.set_index('index').transpose().reset_index(drop = False)

In [54]:
data.head()

index,NAME,Year,Occupation,Employment,Percent of Employment
0,United States,2013,Occ:Total,144255264.57775,NaN
1,Tennessee,2013,Occ:Total,2942359.335027,NaN
2,MPO,2013,Occ:Total,838877.375595,NaN
3,GNRC,2013,Occ:Total,897833.533752,NaN
4,GNRC Region,2013,Occ:Total,929693.931695,NaN


In [55]:
#go long to wide
data = pd.pivot(data, index = ['NAME', 'Year'], columns = 'Occupation', values = ['Employment', 'Percent of Employment'])

In [56]:
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
data.columns = data.columns.to_flat_index()

In [57]:
#rename
occdict = {('Employment', 'Occ:Business & Financial'): 'Occ:Business & Financial', 
           ('Employment', 'Occ:Computer, Mathematical, & Engineering'): 'Occ:Computer, Mathematical, & Engineering',
           ('Employment', 'Occ:Education, Legal, Social, Arts, & Media'): 'Occ:Education, Legal, Social, Arts, & Media', 
           ('Employment', 'Occ:Food, Property, & Personal Services'): 'Occ:Food, Property, & Personal Services', 
           ('Employment', 'Occ:Healthcare Practitioners & Technical'): 'Occ:Healthcare Practitioners & Technical', 
           ('Employment', 'Occ:Healthcare Support'): 'Occ:Healthcare Support', 
           ('Employment', 'Occ:Natural Resources, Construction, & Maintenance'): 'Occ:Natural Resources, Construction, & Maintenance', 
           ('Employment', 'Occ:Production'): 'Occ:Production', 
           ('Employment', 'Occ:Protective Services'): 'Occ:Protective Services', 
           ('Employment', 'Occ:Sales & Office Support'): 'Occ:Sales & Office Support', 
           ('Employment', 'Occ:Total'): 'Occ:Total', 
           ('Employment', 'Occ:Transportation & Material Moving'): 'Occ:Transportation & Material Moving',
           ('Percent of Employment', 'Occ:Transportation & Material Moving'): 'Occ%:Transportation & Material Moving', 
           ('Percent of Employment', 'Occ:Business & Financial'): 'Occ%:Business & Financial', 
           ('Percent of Employment', 'Occ:Computer, Mathematical, & Engineering'): 'Occ%:Computer, Mathematical, & Engineering',
           ('Percent of Employment', 'Occ:Education, Legal, Social, Arts, & Media'): 'Occ%:Education, Legal, Social, Arts, & Media', 
           ('Percent of Employment', 'Occ:Food, Property, & Personal Services'): 'Occ%:Food, Property, & Personal Services', 
           ('Percent of Employment', 'Occ:Healthcare Practitioners & Technical'): 'Occ%:Healthcare Practitioners & Technical', 
           ('Percent of Employment', 'Occ:Healthcare Support'): 'Occ%:Healthcare Support', 
           ('Percent of Employment', 'Occ:Natural Resources, Construction, & Maintenance'): 'Occ%:Natural Resources, Construction, & Maintenance', 
           ('Percent of Employment', 'Occ:Production'): 'Occ%:Production', 
           ('Percent of Employment', 'Occ:Protective Services'): 'Occ%:Protective Services', 
           ('Percent of Employment', 'Occ:Sales & Office Support'): 'Occ%:Sales & Office Support', 
           ('Percent of Employment', 'Occ:Total'): 'Occ%:Total', 
           ('Percent of Employment', 'Occ:Transportation & Material Moving'): 'Occ%:Transportation & Material Moving' }
data = data.rename(columns = occdict)
data = data.reset_index(drop = False)

In [58]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [59]:
data.head(2)

,NAME,Year,Occ:Business & Financial,"Occ:Computer, Mathematical, & Engineering","Occ:Education, Legal, Social, Arts, & Media","Occ:Food, Property, & Personal Services",Occ:Healthcare Practitioners & Technical,Occ:Healthcare Support,"Occ:Natural Resources, Construction, & Maintenance",Occ:Production,Occ:Protective Services,Occ:Sales & Office Support,Occ:Total,Occ:Transportation & Material Moving,Occ%:Business & Financial,"Occ%:Computer, Mathematical, & Engineering","Occ%:Education, Legal, Social, Arts, & Media","Occ%:Food, Property, & Personal Services",Occ%:Healthcare Practitioners & Technical,Occ%:Healthcare Support,"Occ%:Natural Resources, Construction, & Maintenance",Occ%:Production,Occ%:Protective Services,Occ%:Sales & Office Support,Occ%:Total,Occ%:Transportation & Material Moving
0,"Adams city, Tennessee",2013,4.391654,0.63771,6.699578,5.791838,0.812895,3.027161,3.774192,0.512623,1.276359,20.901959,54.305139,6.47917,8.086996,1.174309,12.336913,10.66536,1.496902,5.574354,6.949972,0.943967,2.350346,38.489836,NaN,11.931045
1,"Adams city, Tennessee",2014,4.521009,0.656609,6.110289,12.050281,0.898256,3.684721,3.396287,0.744476,1.060097,19.48105,59.592414,6.989339,7.586551,1.101832,10.253468,20.221167,1.507333,6.183205,5.699193,1.249279,1.778913,32.690486,NaN,11.728572


In [60]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [61]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Occ%:Business & Financial                                                                                    \
Year                                              2013      2014       2015       2016      2017      2018     2019      2020       2021   
NAME                                                                                                                                       
Adams city, Tennessee                         8.086996  7.586551   7.979162   8.371185  8.963427  9.381756  9.27792  9.734560  10.757221   
Ashland City town, Tennessee                  9.511622  9.933443  10.892460  10.166274  9.327253  9.448122  9.65157  9.811132  10.290647   

                             Occ%:Computer, Mathematical, & Engineering                                                                        \
Year                                                               2013      2014      2015      2016      2017      2018      2019      2020   
NAME                                                                                                                                            
Adams city, Tennessee                                          1.174309  1.101832  1.127327  1.380867  1.448803  1.330971  1.291932  1.413762   
Ashland City town, Tennessee                                   4.331725  4.546615  4.844441  4.743846  4.517522  4.646922  4.831349  5.023975   

                                       Occ%:Education, Legal, Social, Arts, & Media                                                                \
Year                              2021                                         2013       2014      2015      2016      2017      2018       2019   
NAME                                                                                                                                                
Adams city, Tennessee         1.544138                                    12.336913  10.253468  9.710685  9.602445  9.885555  9.979423  10.329009   
Ashland City town, Tennessee  5.156669                                     5.339779   5.404677  5.274097  4.882854  4.379309  4.528757   4.444668   

                                                   Occ%:Food, Property, & Personal Services                                                         \
Year                               2020       2021                                     2013       2014       2015       2016       2017       2018   
NAME                                                                                                                                                 
Adams city, Tennessee         12.759787  13.959494                                10.665360  20.221167  25.782003  22.818909  19.298882  19.432404   
Ashland City town, Tennessee   4.188445   3.958277                                 9.547983   9.828160   9.355322   9.638699  10.881262  11.303546   

                                                              Occ%:Healthcare Practitioners & Technical                                          \
Year                               2019       2020       2021                                      2013      2014      2015      2016      2017   
NAME                                                                                                                                              
Adams city, Tennessee         19.599968  18.757045  14.856289                                  1.496902  1.507333  2.077365  3.202943  3.889906   
Ashland City town, Tennessee  10.988355  10.117011   9.677044                                  1.435606  1.605688  1.431218  1.818764  2.022886   

                                                                     Occ%:Healthcare Support                                                    \
Year                              2018      2019      2020      2021                    2013      2014      2015      2016      2017      2018   
NAME                                                                         

In [62]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Occ%:Business & Financial                                                                                     \
                                                  2013      2014       2015       2016      2017      2018      2019      2020       2021   
                                                  None      None       None       None      None      None      None      None       None   
NAME                                                                                                                                        
Adams city, Tennessee                         8.086996  7.586551   7.979162   8.371185  8.963427  9.381756  9.277920  9.734560  10.757221   
Ashland City town, Tennessee                  9.511622  9.933443  10.892460  10.166274  9.327253  9.448122  9.651570  9.811132  10.290647   
Belle Meade city, Tennessee                   7.768709  8.151045   8.134494   8.154676  8.533155  8.919130  9.100849  9.446925   9.787968   

                             Occ%:Computer, Mathematical, & Engineering                                                                        \
                                                                   2013      2014      2015      2016      2017      2018      2019      2020   
                                                                   None      None      None      None      None      None      None      None   
NAME                                                                                                                                            
Adams city, Tennessee                                          1.174309  1.101832  1.127327  1.380867  1.448803  1.330971  1.291932  1.413762   
Ashland City town, Tennessee                                   4.331725  4.546615  4.844441  4.743846  4.517522  4.646922  4.831349  5.023975   
Belle Meade city, Tennessee                                    1.513198  1.565971  1.356673  1.269079  1.333570  1.392946  1.407307  1.534650   

                                       Occ%:Education, Legal, Social, Arts, & Media                                                        \
                                  2021                                         2013       2014       2015       2016       2017      2018   
                                  None                                         None       None       None       None       None      None   
NAME                                                                                                                                        
Adams city, Tennessee         1.544138                                    12.336913  10.253468   9.710685   9.602445   9.885555  9.979423   
Ashland City town, Tennessee  5.156669                                     5.339779   5.404677   5.274097   4.882854   4.379309  4.528757   
Belle Meade city, Tennessee   1.655984                                    12.961993  12.602382  11.729883  11.498497  10.834219  9.827061   

                                                              Occ%:Food, Property, & Personal Services                                              \
                                   2019       2020       2021                                     2013       2014       2015       2016       2017   
                                   None       None       None                                     None       None       None       None       None   
NAME                                                                                                                                                 
Adams city, Tennessee         10.329009  12.759787  13.959494                                10.665360  20.221167  25.782003  22.818909  19.298882   
Ashland City town, Tennessee   4.444668   4.188445   3.958277                                 9.547983   9.828160   9.355322   9.638699  10.881262   
Belle Meade city, Tennessee    9.336153   9.087959   8.863774                                38.983969  39.568668  41.620139  43.126687  

In [63]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [64]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [65]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [66]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [67]:
#final check
data.head(3)

,NAME,Year,Time Frame,Occ%:Business & Financial,"Occ%:Computer, Mathematical, & Engineering","Occ%:Education, Legal, Social, Arts, & Media","Occ%:Food, Property, & Personal Services",Occ%:Healthcare Practitioners & Technical,Occ%:Healthcare Support,"Occ%:Natural Resources, Construction, & Maintenance",Occ%:Production,Occ%:Protective Services,Occ%:Sales & Office Support,Occ%:Transportation & Material Moving,Occ:Business & Financial,Occ:Business & Financial % Change,Occ:Business & Financial Change,"Occ:Computer, Mathematical, & Engineering","Occ:Computer, Mathematical, & Engineering % Change","Occ:Computer, Mathematical, & Engineering Change","Occ:Education, Legal, Social, Arts, & Media","Occ:Education, Legal, Social, Arts, & Media % Change","Occ:Education, Legal, Social, Arts, & Media Change","Occ:Food, Property, & Personal Services","Occ:Food, Property, & Personal Services % Change","Occ:Food, Property, & Personal Services Change",Occ:Healthcare Practitioners & Technical,Occ:Healthcare Practitioners & Technical % Change,Occ:Healthcare Practitioners & Technical Change,Occ:Healthcare Support,Occ:Healthcare Support % Change,Occ:Healthcare Support Change,"Occ:Natural Resources, Construction, & Maintenance","Occ:Natural Resources, Construction, & Maintenance % Change","Occ:Natural Resources, Construction, & Maintenance Change",Occ:Production,Occ:Production % Change,Occ:Production Change,Occ:Protective Services,Occ:Protective Services % Change,Occ:Protective Services Change,Occ:Sales & Office Support,Occ:Sales & Office Support % Change,Occ:Sales & Office Support Change,Occ:Total,Occ:Total % Change,Occ:Total Change,Occ:Transportation & Material Moving,Occ:Transportation & Material Moving % Change,Occ:Transportation & Material Moving Change,GEO_ID,Source
0,"Adams city, Tennessee",2013,None,8.086996,1.174309,12.336913,10.665360,1.496902,5.574354,6.949972,0.943967,2.350346,38.489836,11.931045,4.391654,NaN,NaN,0.637710,NaN,NaN,6.699578,NaN,NaN,5.791838,NaN,NaN,0.812895,NaN,NaN,3.027161,NaN,NaN,3.774192,NaN,NaN,0.512623,NaN,NaN,1.276359,NaN,NaN,20.901959,NaN,NaN,54.305139,NaN,NaN,6.479170,NaN,NaN,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2014,None,7.586551,1.101832,10.253468,20.221167,1.507333,6.183205,5.699193,1.249279,1.778913,32.690486,11.728572,4.521009,NaN,NaN,0.656609,NaN,NaN,6.110289,NaN,NaN,12.050281,NaN,NaN,0.898256,NaN,NaN,3.684721,NaN,NaN,3.396287,NaN,NaN,0.744476,NaN,NaN,1.060097,NaN,NaN,19.481050,NaN,NaN,59.592414,NaN,NaN,6.989339,NaN,NaN,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2015,None,7.979162,1.127327,9.710685,25.782003,2.077365,5.731286,7.095432,1.081058,2.286490,27.554628,9.574563,5.393061,NaN,NaN,0.761952,NaN,NaN,6.563385,NaN,NaN,17.425878,NaN,NaN,1.404077,NaN,NaN,3.873737,NaN,NaN,4.795753,NaN,NaN,0.730680,NaN,NaN,1.545423,NaN,NaN,18.623983,NaN,NaN,67.589310,NaN,NaN,6.471381,NaN,NaN,1600000US4700200,JobsEQ


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3870 entries, 0 to 3869
Data columns (total 52 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   NAME                                                         3870 non-null   object 
 1   Year                                                         3870 non-null   object 
 2   Time Frame                                                   3870 non-null   object 
 3   Occ%:Business & Financial                                    774 non-null    float64
 4   Occ%:Computer, Mathematical, & Engineering                   774 non-null    float64
 5   Occ%:Education, Legal, Social, Arts, & Media                 774 non-null    float64
 6   Occ%:Food, Property, & Personal Services                     774 non-null    float64
 7   Occ%:Healthcare Practitioners & Technical                    774 non-null    f

In [69]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_OccupationalEmployment_Annual_Change', conn, if_exists = 'replace', index = False)

3870

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function 
+ *load the following query: JobsEQ Annual Datasets: JobsEQ_IndustryEmployment_Annual_Change*

The guidance for formatting this query manually is below:  
+ Select the "Industry" dataset, annual  
+ Select the following dimensions: Region, Industry, Year/Quarter, then select "Employment" as the value  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Census Places"   
+ For "Year/Quarter" select as many as possible  
+ Under "Industry" expand "Total-All Industries" and select the combination of 2 digit NAICS codes and built groups in Jobs EQ as described below, leaving Total as well  

Industries to Select:  
+ Total - All Industries  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Snapshot 11: Construction (23: Construction)  
+ Snapshot 11: Manufacturing (31: Manufacturing)  
+ Snapshot 11: Information (51: Information)  
+ Snapshot 11: Public Administration (92: Public Administration)  
+ Snapshot 11: Other/Unclassified (81: Other, Except Public Administration, 99: Unclassified)  
+ Snapshot 11: Leisure & Hospitality (71: Arts, Entertainment, & Recreation, 72: Accommodation & Food Services)   
+ Snapshot 11: Education & Health Services (61: Education, 62: Healthcare & Social Assistance)   
+ Snapshot 11: Professional & Business Services (54: Professional, Scientific, & Technical, 55: Management of Companies & Enterprises, 56: Administrative & Support & Waste Management & Remediation)   
+ Snapshot 11: Financial Activities (52: Finance & Insurance, 53: Real Estate & Rental & Leasing)   
+ Snapshot 11: Trade, Transportation & Utilities (42: Wholesale Trade, 44-45: Retail Trade, 48-49: Transportation & Warehousing, 22: Utilities)  
+ Snapshot 11: Natural Resources & Mining (11: Agricultural Activities, Forestry, Fishing, & Hunting, 21: Mining, Quarrying, Oil & Gas) 
+ Snapshot: White Collar  
+ Snapshot: Blue Collar  
+ Snapshot 4: Government  
+ Snapshot 4: Retail & Hospitality  
+ Snapshot 4: Professional Services  
+ Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics  

Select: 'Employment'

This may exceed the drafting size, in which case you'll have to queue for download. Make sure Industry is on the top and Region and Time are on the left side. 

*After loading the query and downloading, resume here*  

Once it is completed, open and delete the top row that says "Industry Data", and delete the third row that repeats "Empl". Delete the "FIPS" column, change "Time" to "Year", and "Region" to "NAME". Select all numerical data and format to "general". Delete metadata at the bottom. Save in the "Data Downloads" folder as "JobsEQ_IndustryEmploymentAnnual_AllGeos.csv"

In [130]:
#import and examine, dtype str so the GEO_ID formatting doesn't get messed up and so that we can split the time column
data = pd.read_csv('../../Data Downloads/JobsEQ_IndustryEmploymentAnnual_AllGeos.csv', dtype = str)
data.head(3)

,NAME,Year,Total - All Industries,Construction (23),Manufacturing (31),Information (51),Public Administration (92),Other/Unclassified,Leisure & Hospitality,Education & Health Services,Professional & Business Services,Financial Activities,"Trade, Transportation, & Utilities",Natural Resources & Mining,Snapshot: Government,Snapshot: Blue Collar,Snapshot: White Collar,Snapshot: Retail & Hospitality,Snapshot: Professional Services,"Snapshot: Industrial, Manufacturing, Utilities, & Logistics"
0,USA,2001,141312101.3,8912185.963,16739187.99,3878271.943,7041156.101,6860270.75,12834181.69,26555118.85,18317200.74,8600864.635,28716338.97,2857323.615,7041156.101,38835049.19,55852067.73,26428516.27,66703836.54,41138592.33
1,USA,2002,139809572.2,8780883.527,15557008.68,3651348.546,7137264.899,6923862.558,12947415.61,27254064.05,17898069.6,8612551.638,28222178.08,2824925.038,7137264.899,37178601.1,56175860.96,26343616.72,66849332.26,39479358.34
2,USA,2003,139773343.4,8857297.819,14822303.47,3473713.055,7147366.318,7017371.562,13158602.83,27785568.38,17891966.19,8770955.81,28073378.65,2774819.269,7147366.318,36417115.53,56690961.57,26461914.57,67491299.53,38672762.95


In [131]:
#quickly rename to full census geographies
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [132]:
#make a list of years to loop through grouping incorporated and unincorporated areas
times = list(data['Year'].unique())

In [133]:
data = data.set_index(['NAME', 'Year']).transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [134]:
#aggregate by double column-headers
for time in times:
    RuthInc = [data[('Eagleville city, Tennessee', '{}'.format(time))],data[('La Vergne city, Tennessee', '{}'.format(time))],
               data[('Murfreesboro city, Tennessee', '{}'.format(time))],data[('Smyrna town, Tennessee', '{}'.format(time))]]
    data[('Rutherford Incorporated', '{}'.format(time))] = sum(RuthInc)
    data[('Rutherford Unincorporated', '{}'.format(time))] = data[('Rutherford County, Tennessee', '{}'.format(time))] - data[('Rutherford Incorporated', '{}'.format(time))]
    WilsonInc = [data[('Lebanon city, Tennessee', '{}'.format(time))],data[('Mount Juliet city, Tennessee', '{}'.format(time))],
                 data[('Watertown city, Tennessee', '{}'.format(time))]]
    data[('Wilson Incorporated', '{}'.format(time))] = sum(WilsonInc)
    data[('Wilson Unincorporated', '{}'.format(time))] = data[('Wilson County, Tennessee', '{}'.format(time))] - data[('Wilson Incorporated', '{}'.format(time))]
    CheathInc = [data[('Ashland City town, Tennessee', '{}'.format(time))],data[('Kingston Springs town, Tennessee', '{}'.format(time))],
                 data[('Pegram town, Tennessee', '{}'.format(time))],data[('Pleasant View city, Tennessee', '{}'.format(time))]]
    data[('Cheatham Incorporated', '{}'.format(time))] = sum(CheathInc)
    data[('Cheatham Unincorporated', '{}'.format(time))] = data[('Cheatham County, Tennessee', '{}'.format(time))] - data[('Cheatham Incorporated', '{}'.format(time))]
    DicksInc = [data[('Burns town, Tennessee', '{}'.format(time))],data[('Charlotte town, Tennessee', '{}'.format(time))],
                data[('Dickson city, Tennessee', '{}'.format(time))],data[('Slayden town, Tennessee', '{}'.format(time))],
                data[('Vanleer town, Tennessee', '{}'.format(time))],data[('White Bluff town, Tennessee', '{}'.format(time))]]
    data[('Dickson Incorporated', '{}'.format(time))] = sum(DicksInc)
    data[('Dickson Unincorporated', '{}'.format(time))] = data[('Dickson County, Tennessee', '{}'.format(time))] - data[('Dickson Incorporated', '{}'.format(time))]
    HumphInc = [data[('McEwen city, Tennessee', '{}'.format(time))],data[('New Johnsonville city, Tennessee', '{}'.format(time))],
                data[('Waverly city, Tennessee', '{}'.format(time))]]
    data[('Humphreys Incorporated', '{}'.format(time))] = sum(HumphInc)
    data[('Humphreys Unincorporated', '{}'.format(time))] = data[('Humphreys County, Tennessee', '{}'.format(time))] - data[('Humphreys Incorporated', '{}'.format(time))]
    data[('Montgomery Incorporated', '{}'.format(time))] = data[('Clarksville city, Tennessee', '{}'.format(time))]
    data[('Montgomery Unincorporated', '{}'.format(time))] = data[('Montgomery County, Tennessee', '{}'.format(time))] - data[('Montgomery Incorporated', '{}'.format(time))]

In [135]:
data = data.transpose()
data.head()

Total - All Industries  Construction (23)  Manufacturing (31)  Information (51)  Public Administration (92)  Other/Unclassified  \
NAME          Year                                                                                                                                    
United States 2001             141312101.3        8912185.963         16739187.99       3878271.943                 7041156.101         6860270.750   
              2002             139809572.2        8780883.527         15557008.68       3651348.546                 7137264.899         6923862.558   
              2003             139773343.4        8857297.819         14822303.47       3473713.055                 7147366.318         7017371.562   
              2004             141413689.2        9124560.364         14613050.29       3395316.103                 7118886.819         7108144.569   
              2005             143716164.0        9514665.412         14528058.04       3352728.004                 7149281.864         7129697.232   

                    Leisure & Hospitality  Education & Health Services  Professional & Business Services  Financial Activities  \
NAME          Year                                                                                                               
United States 2001            12834181.69                  26555118.85                       18317200.74           8600864.635   
              2002            12947415.61                  27254064.05                       17898069.60           8612551.638   
              2003            13158602.83                  27785568.38                       17891966.19           8770955.810   
              2004            13486792.15                  28234216.82                       18365802.95           8844618.848   
              2005            13774790.90                  28774737.34                       18969862.55           9030091.901   

                    Trade, Transportation, & Utilities  Natural Resources & Mining  Snapshot: Government  Snapshot: Blue Collar  \
NAME          Year                                                                                                                
United States 2001                         28716338.97                 2857323.615           7041156.101            38835049.19   
              2002                         28222178.08                 2824925.038           7137264.899            37178601.10   
              2003                         28073378.65                 2774819.269           7147366.318            36417115.53   
              2004                         28313599.92                 2808700.409           7118886.819            36595373.19   
              2005                         28646807.06                 2845443.688           7149281.864            37116021.64   

                    Snapshot: White Collar  Snapshot: Retail & Hospitality  Snapshot: Professional Services  \
NAME          Year                                                                                            
United States 2001             55852067.73                     26428516.27                      66703836.54   
              2002             56175860.96                     26343616.72                      66849332.26   
              2003             56690961.57                     26461914.57                      67491299.53   
              2004             57298470.50                     26879000.65                      68551023.99   
              2005             58354344.69                     27297902.35                      69889087.71   

                    Snapshot: Industrial, Manufacturing, Utilities, & Logistics  
NAME          Year                                                               
United States 2001                                        41138592.33            
              2002                                        39479358.34            
              2003         

In [136]:
#sum these, note that this categorization for service producing doesn't contain trade, transportation & utilities as that's broken out separately
thelist = [data['Natural Resources & Mining'], data['Construction (23)'], data['Manufacturing (31)']]
data['Goods Producing'] = sum(thelist)
thelist = [data['Information (51)'], data['Financial Activities'], data['Professional & Business Services'], 
           data['Education & Health Services'], data['Leisure & Hospitality'], data['Public Administration (92)'], data['Other/Unclassified']]
data['Service Producing'] = sum(thelist)

In [137]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Construction (23)'], data['Manufacturing (31)'], 
           data['Information (51)'], data['Public Administration (92)'], 
           data['Other/Unclassified'], data['Leisure & Hospitality'],
           data['Education & Health Services'], data['Professional & Business Services'], 
           data['Financial Activities'], data['Trade, Transportation, & Utilities'], data['Natural Resources & Mining']]
data['Total - All Industries'] = sum(thelist)

In [138]:
#rename columns to account for the disparity between custom groups and NAICS 2 digit formatting
data = data.rename(columns = {'Total - All Industries': 'Total', 'Construction (23)': 'Construction',
                   'Manufacturing (31)': 'Manufacturing', 'Information (51)': 'Information', 'Public Administration (92)': 'Public Administration'})

In [139]:
#set the index to year and name so that we can add a prefix to all of the other columns, before resetting the index without dropping
data.reset_index(drop = False, inplace = True)
data = data.set_index(['NAME', 'Year']).add_prefix('Ind:').reset_index(drop = False)

In [140]:
#melt this data down using the columns name and year to distinguish specific instances, and then attributing total employment per occupation
#name it separately because it bugs out if you call this data and merge later
cols = data.columns
thing = data.melt(var_name = 'Industry', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Employment')
thing.head(2)

,NAME,Year,Industry,Employment
0,United States,2001,Ind:Total,1.413121e+08
1,United States,2002,Ind:Total,1.398096e+08


In [141]:
#create a separate dataframe where we will calculate percent of the total for each occupation group
percents = data

In [142]:
#calculate percent of total
percents['Ind:Construction'] = percent(percents['Ind:Construction'], percents['Ind:Total'])
percents['Ind:Manufacturing'] = percent(percents['Ind:Manufacturing'], percents['Ind:Total'])
percents['Ind:Information'] = percent(percents['Ind:Information'], percents['Ind:Total'])
percents['Ind:Public Administration'] = percent(percents['Ind:Public Administration'], percents['Ind:Total'])
percents['Ind:Other/Unclassified'] = percent(percents['Ind:Other/Unclassified'], percents['Ind:Total'])
percents['Ind:Leisure & Hospitality'] = percent(percents['Ind:Leisure & Hospitality'], percents['Ind:Total'])
percents['Ind:Education & Health Services'] = percent(percents['Ind:Education & Health Services'], percents['Ind:Total'])
percents['Ind:Professional & Business Services'] = percent(percents['Ind:Professional & Business Services'], percents['Ind:Total'])
percents['Ind:Financial Activities'] = percent(percents['Ind:Financial Activities'], percents['Ind:Total'])
percents['Ind:Trade, Transportation, & Utilities'] = percent(percents['Ind:Trade, Transportation, & Utilities'], percents['Ind:Total'])
percents['Ind:Natural Resources & Mining'] = percent(percents['Ind:Natural Resources & Mining'], percents['Ind:Total'])
percents['Ind:Goods Producing'] = percent(percents['Ind:Goods Producing'], percents['Ind:Total'])
percents['Ind:Service Producing'] = percent(percents['Ind:Service Producing'], percents['Ind:Total'])
percents['Ind:Snapshot: Government'] = percent(percents['Ind:Snapshot: Government'], percents['Ind:Total'])
percents['Ind:Snapshot: Blue Collar'] = percent(percents['Ind:Snapshot: Blue Collar'], percents['Ind:Total'])
percents['Ind:Snapshot: White Collar'] = percent(percents['Ind:Snapshot: White Collar'], percents['Ind:Total'])
percents['Ind:Snapshot: Retail & Hospitality'] = percent(percents['Ind:Snapshot: Retail & Hospitality'], percents['Ind:Total'])
percents['Ind:Snapshot: Professional Services'] = percent(percents['Ind:Snapshot: Professional Services'], percents['Ind:Total'])
percents['Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'] = percent(percents['Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'], 
                                                                                      percents['Ind:Total'])

In [143]:
#similarly melt this dataframe
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Industry', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Percent of Employment')
percents.head()

,NAME,Year,Industry,Percent of Employment
0,United States,2001,Ind:Total,1.413121e+08
1,United States,2002,Ind:Total,1.398096e+08
2,United States,2003,Ind:Total,1.397733e+08
3,United States,2004,Ind:Total,1.414137e+08
4,United States,2005,Ind:Total,1.437162e+08


In [144]:
#we never renamed the total column when we duplicated the dataframe for percents, we want this to be null so just transpose, drop it, and transpose back
percents = percents.set_index('Industry').transpose()
percents = percents.drop(columns = 'Ind:Total')
percents = percents.transpose().reset_index(drop = False)

In [145]:
#merge total employment and percent of total employment
data = thing.merge(percents, on = ['Industry', 'Year', 'NAME'], how = 'outer')
data.head(3)

,NAME,Year,Industry,Employment,Percent of Employment
0,United States,2001,Ind:Total,1.413121e+08,NaN
1,United States,2002,Ind:Total,1.398096e+08,NaN
2,United States,2003,Ind:Total,1.397733e+08,NaN


In [146]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos beforeresetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose().reset_index(drop = False)

In [147]:
#set the index to "index" (the variable names) and transpose, then reset the index without dropping it
data = data.set_index('index').transpose().reset_index(drop = False)

In [148]:
data.head()

index,NAME,Year,Industry,Employment,Percent of Employment
0,United States,2001,Ind:Total,141312101.247,NaN
1,United States,2002,Ind:Total,139809572.226,NaN
2,United States,2003,Ind:Total,139773343.353,NaN
3,United States,2004,Ind:Total,141413689.242,NaN
4,United States,2005,Ind:Total,143716163.991,NaN


In [149]:
#go long to wide
data = pd.pivot(data, index = ['NAME', 'Year'], columns = 'Industry', values = ['Employment', 'Percent of Employment'])

In [150]:
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
data.columns = data.columns.to_flat_index()

In [151]:
#rename
inddict = {('Employment', 'Ind:Education & Health Services'): 'Ind:Education & Health Services', 
           ('Employment', 'Ind:Construction'): 'Ind:Construction', 
           ('Employment', 'Ind:Financial Activities'): 'Ind:Financial Activities',
           ('Employment', 'Ind:Information'): 'Ind:Information', 
           ('Employment', 'Ind:Manufacturing'): 'Ind:Manufacturing', 
           ('Employment', 'Ind:Other/Unclassified'): 'Ind:Other/Unclassified', 
           ('Employment', 'Ind:Public Administration'): 'Ind:Public Administration', 
           ('Employment', 'Ind:Total'): 'Ind:Total', 
           ('Employment', 'Ind:Goods Producing'): 'Ind:Goods Producing', 
           ('Employment', 'Ind:Leisure & Hospitality'): 'Ind:Leisure & Hospitality', 
           ('Employment', 'Ind:Natural Resources & Mining'): 'Ind:Natural Resources & Mining', 
           ('Employment', 'Ind:Professional & Business Services'): 'Ind:Professional & Business Services', 
           ('Employment', 'Ind:Service Producing'): 'Ind:Service Producing', 
           ('Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind:Trade, Transportation, & Utilities',
           ('Employment', 'Ind:Snapshot: Government'): 'Ind:Snapshot: Government',
           ('Employment', 'Ind:Snapshot: Blue Collar'): 'Ind:Snapshot: Blue Collar',
           ('Employment', 'Ind:Snapshot: White Collar'): 'Ind:Snapshot: White Collar',
           ('Employment', 'Ind:Snapshot: Retail & Hospitality'): 'Ind:Snapshot: Retail & Hospitality',
           ('Employment', 'Ind:Snapshot: Professional Services'): 'Ind:Snapshot: Professional Services',
           ('Employment', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics',
           ('Percent of Employment', 'Ind:Education & Health Services'): 'Ind%:Education & Health Services',
           ('Percent of Employment', 'Ind:Construction'): 'Ind%:Construction', 
           ('Percent of Employment', 'Ind:Financial Activities'): 'Ind%:Financial Activities',
           ('Percent of Employment', 'Ind:Information'): 'Ind%:Information', 
           ('Percent of Employment', 'Ind:Manufacturing'): 'Ind%:Manufacturing', 
           ('Percent of Employment', 'Ind:Other/Unclassified'): 'Ind%:Other/Unclassified', 
           ('Percent of Employment', 'Ind:Public Administration'): 'Ind%:Public Administration', 
           ('Percent of Employment', 'Ind:Total'): 'Ind%:Total', 
           ('Percent of Employment', 'Ind:Goods Producing'): 'Ind%:Goods Producing', 
           ('Percent of Employment', 'Ind:Leisure & Hospitality'): 'Ind%:Leisure & Hospitality', 
           ('Percent of Employment', 'Ind:Natural Resources & Mining'): 'Ind%:Natural Resources & Mining', 
           ('Percent of Employment', 'Ind:Professional & Business Services'): 'Ind%:Professional & Business Services', 
           ('Percent of Employment', 'Ind:Service Producing'): 'Ind%:Service Producing', 
           ('Percent of Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind%:Trade, Transportation, & Utilities', 
           ('Percent of Employment', 'Ind:Snapshot: Government'): 'Ind%:Snapshot: Government',
           ('Percent of Employment', 'Ind:Snapshot: Blue Collar'): 'Ind%:Snapshot: Blue Collar',
           ('Percent of Employment', 'Ind:Snapshot: White Collar'): 'Ind%:Snapshot: White Collar',
           ('Percent of Employment', 'Ind:Snapshot: Retail & Hospitality'): 'Ind%:Snapshot: Retail & Hospitality',
           ('Percent of Employment', 'Ind:Snapshot: Professional Services'): 'Ind%:Snapshot: Professional Services',
           ('Percent of Employment', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'
           }
data = data.rename(columns = inddict)
data = data.reset_index(drop = False)

In [152]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [153]:
data.head(2)

,NAME,Year,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Snapshot: Blue Collar,Ind:Snapshot: Government,"Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind:Snapshot: Professional Services,Ind:Snapshot: Retail & Hospitality,Ind:Snapshot: White Collar,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: Government,"Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot: Professional Services,Ind%:Snapshot: Retail & Hospitality,Ind%:Snapshot: White Collar,Ind%:Total,"Ind%:Trade, Transportation, & Utilities"
0,"Adams city, Tennessee",2001,0.426199,0.273155,0.25816,2.182215,0.01973,0.212292,0.001641,1.754376,2.511192,0.772678,15.572745,19.619952,11.803753,15.572745,13.558129,4.047178,2.347142,16.784326,35.525195,13.723027,1.199708,0.768904,0.726697,6.142726,0.055537,0.597581,0.004619,4.938398,7.068764,2.175015,43.835777,55.228275,33.226428,43.835777,38.164827,11.392417,6.606979,47.246261,NaN,38.629
1,"Adams city, Tennessee",2002,0.407432,0.267267,0.253642,2.148894,0.016119,0.207917,0.003787,1.737675,2.12713,0.754309,16.908913,20.535298,12.315958,16.908913,14.053552,3.626356,2.437756,18.090371,37.026577,14.342385,1.100377,0.721826,0.685028,5.803653,0.043533,0.561535,0.010228,4.693048,5.744872,2.037211,45.666964,55.460967,33.262479,45.666964,37.955309,9.793928,6.5838,48.8578,NaN,38.735379


In [154]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [155]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Ind%:Construction                                                                                                       \
Year                                      2001      2002      2003      2004      2005       2006      2007      2008      2009      2010      2011   
NAME                                                                                                                                                  
Adams city, Tennessee                 1.199708  1.100377  0.934419  0.848531  0.625128  10.704603  0.492263  0.379313  0.457333  6.606593  7.261369   
Ashland City town, Tennessee          4.759270  3.797778  2.965820  3.398875  3.620457   4.008022  2.294908  2.497255  2.847316  2.696543  3.393320   

                                                                                                                                  \
Year                              2012      2013      2014      2015      2016      2017      2018      2019      2020      2021   
NAME                                                                                                                               
Adams city, Tennessee         0.540998  3.929823  0.427893  6.265321  2.377485  2.267936  2.611494  0.553936  0.506698  0.511350   
Ashland City town, Tennessee  3.709080  3.934544  4.239339  2.108557  1.891799  2.007638  2.107879  3.477568  3.396544  3.404569   

                             Ind%:Education & Health Services                                                                                  \
Year                                                     2001      2002      2003      2004      2005      2006      2007      2008      2009   
NAME                                                                                                                                            
Adams city, Tennessee                                0.768904  0.721826  0.599388  0.527586  0.405393  0.553558  0.515809  0.322339  0.380556   
Ashland City town, Tennessee                        11.019619  9.046620  9.105081  6.825123  6.654594  5.855103  6.415881  6.140544  6.728626   

                                                                                                                                           \
Year                              2010       2011       2012       2013       2014       2015       2016       2017       2018       2019   
NAME                                                                                                                                        
Adams city, Tennessee         0.422762  19.700286  23.838074  27.553884  25.084066  22.408145  22.189224  24.517513  26.389973  29.846975   
Ashland City town, Tennessee  6.935628   6.670558   7.123102   7.477139   7.636647   6.035749   6.863534   5.904690   5.351894   5.005060   

                                                   Ind%:Financial Activities                                                                \
Year                               2020       2021                      2001      2002      2003      2004       2005      2006       2007   
NAME                                                                                                                                         
Adams city, Tennessee         33.087883  35.260019                  0.726697  0.685028  2.075514  0.410591  22.223826  9.901537  11.219139   
Ashland City town, Tennessee   5.082416   5.091556                  3.804546  3.158668  3.438730  3.378791   3.114758  2.686272   3.542879   

                                                                                                                                            \
Year                              2008      2009      2010      2011      2012      2013      2014      2015      2016      2017      2018   
NAME                                                                                                                                         
Adams city, Tennessee         7.538294  6.2

In [156]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Ind%:Construction                                                                                                       \
                                          2001      2002      2003      2004      2005       2006      2007      2008      2009      2010      2011   
                                          None      None      None      None      None       None      None      None      None      None      None   
NAME                                                                                                                                                  
Adams city, Tennessee                 1.199708  1.100377  0.934419  0.848531  0.625128  10.704603  0.492263  0.379313  0.457333  6.606593  7.261369   
Ashland City town, Tennessee          4.759270  3.797778  2.965820  3.398875  3.620457   4.008022  2.294908  2.497255  2.847316  2.696543  3.393320   
Belle Meade city, Tennessee           1.071348  1.008619  1.568837  1.101413  1.022168   2.655818  1.450226  1.572809  1.303129  1.667614  3.084410   

                                                                                                                                  \
                                  2012      2013      2014      2015      2016      2017      2018      2019      2020      2021   
                                  None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                               
Adams city, Tennessee         0.540998  3.929823  0.427893  6.265321  2.377485  2.267936  2.611494  0.553936  0.506698  0.511350   
Ashland City town, Tennessee  3.709080  3.934544  4.239339  2.108557  1.891799  2.007638  2.107879  3.477568  3.396544  3.404569   
Belle Meade city, Tennessee   1.135321  1.073689  1.445071  2.907085  1.535676  1.903346  1.829653  1.796443  2.152463  2.113983   

                             Ind%:Education & Health Services                                                                                  \
                                                         2001      2002      2003      2004      2005      2006      2007      2008      2009   
                                                         None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                            
Adams city, Tennessee                                0.768904  0.721826  0.599388  0.527586  0.405393  0.553558  0.515809  0.322339  0.380556   
Ashland City town, Tennessee                        11.019619  9.046620  9.105081  6.825123  6.654594  5.855103  6.415881  6.140544  6.728626   
Belle Meade city, Tennessee                          5.079238  5.350355  4.144871  5.055898  3.262138  3.317140  3.980984  3.826763  2.676374   

                                                                                                                                           \
                                  2010       2011       2012       2013       2014       2015       2016       2017       2018       2019   
                                  None       None       None       None       None       None       None       None       None       None   
NAME                                                                                                                                        
Adams city, Tennessee         0.422762  19.700286  23.838074  27.553884  25.084066  22.408145  22.189224  24.517513  26.389973  29.846975   
Ashland City town, Tennessee  6.935628   6.670558   7.123102   7.477139   7.636647   6.035749   6.863534   5.904690   5.351894   5.005060   
Belle Meade city, Tennessee   3.369239   2.309342   4.907627   5.920157   5.442417   5.476078   5.130500   2.664020   4.570016   3.771165   

                                          

In [157]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
# first_level.remove('NAME')
# first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [158]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [159]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [160]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [161]:
#final check
data.head()

,NAME,Year,Time Frame,Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: Government,"Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot: Professional Services,Ind%:Snapshot: Retail & Hospitality,Ind%:Snapshot: White Collar,"Ind%:Trade, Transportation, & Utilities",Ind:Construction,Ind:Construction % Change,Ind:Construction Change,Ind:Education & Health Services,Ind:Education & Health Services % Change,Ind:Education & Health Services Change,Ind:Financial Activities,Ind:Financial Activities % Change,Ind:Financial Activities Change,Ind:Goods Producing,Ind:Goods Producing % Change,Ind:Goods Producing Change,Ind:Information,Ind:Information % Change,Ind:Information Change,Ind:Leisure & Hospitality,Ind:Leisure & Hospitality % Change,Ind:Leisure & Hospitality Change,Ind:Manufacturing,Ind:Manufacturing % Change,Ind:Manufacturing Change,Ind:Natural Resources & Mining,Ind:Natural Resources & Mining % Change,Ind:Natural Resources & Mining Change,Ind:Other/Unclassified,Ind:Other/Unclassified % Change,Ind:Other/Unclassified Change,Ind:Professional & Business Services,Ind:Professional & Business Services % Change,Ind:Professional & Business Services Change,Ind:Public Administration,Ind:Public Administration % Change,Ind:Public Administration Change,Ind:Service Producing,Ind:Service Producing % Change,Ind:Service Producing Change,Ind:Snapshot: Blue Collar,Ind:Snapshot: Blue Collar % Change,Ind:Snapshot: Blue Collar Change,Ind:Snapshot: Government,Ind:Snapshot: Government % Change,Ind:Snapshot: Government Change,"Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics % Change","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics Change",Ind:Snapshot: Professional Services,Ind:Snapshot: Professional Services % Change,Ind:Snapshot: Professional Services Change,Ind:Snapshot: Retail & Hospitality,Ind:Snapshot: Retail & Hospitality % Change,Ind:Snapshot: Retail & Hospitality Change,Ind:Snapshot: White Collar,Ind:Snapshot: White Collar % Change,Ind:Snapshot: White Collar Change,Ind:Total,Ind:Total % Change,Ind:Total Change,"Ind:Trade, Transportation, & Utilities","Ind:Trade, Transportation, & Utilities % Change","Ind:Trade, Transportation, & Utilities Change",GEO_ID,Source
0,"Adams city, Tennessee",2001,None,1.199708,0.768904,0.726697,6.142726,0.055537,0.597581,0.004619,4.938398,7.068764,2.175015,43.835777,55.228275,33.226428,43.835777,38.164827,11.392417,6.606979,47.246261,38.629000,0.426199,NaN,NaN,0.273155,NaN,NaN,0.258160,NaN,NaN,2.182215,NaN,NaN,0.019730,NaN,NaN,0.212292,NaN,NaN,0.001641,NaN,NaN,1.754376,NaN,NaN,2.511192,NaN,NaN,0.772678,NaN,NaN,15.572745,NaN,NaN,19.619952,NaN,NaN,11.803753,NaN,NaN,15.572745,NaN,NaN,13.558129,NaN,NaN,4.047178,NaN,NaN,2.347142,NaN,NaN,16.784326,NaN,NaN,35.525195,NaN,NaN,13.723027,NaN,NaN,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2002,None,1.100377,0.721826,0.685028,5.803653,0.043533,0.561535,0.010228,4.693048,5.744872,2.037211,45.666964,55.460967,33.262479,45.666964,37.955309,9.793928,6.583800,48.857800,38.735379,0.407432,NaN,NaN,0.267267,NaN,NaN,0.253642,NaN,NaN,2.148894,NaN,NaN,0.016119,NaN,NaN,0.207917,NaN,NaN,0.003787,NaN,NaN,1.737675,NaN,NaN,2.127130,NaN,NaN,0.754309,NaN,NaN,16.908913,NaN,NaN,20.535298,NaN,NaN,12.315958,NaN,NaN,16.908913,NaN,NaN,14.053552,NaN,NaN,3.626356,NaN,NaN,2.437756,NaN,NaN,18.090371,NaN,NaN,37.026577,NaN,NaN,14.342385,NaN,NaN,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2003,None,0.934419,0.599388,2.075514,4.394491,0.043680,3.352714,0.026021,3.434050,4.670017,1.689303,39.264909,51.695525,35.667121,39.264909,39.100991,9.544040,12.090060,43.426205,43.909985,0.436279,Na

In [162]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19866 entries, 0 to 19865
Data columns (total 84 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   NAME                                                                      19866 non-null  object 
 1   Year                                                                      19866 non-null  object 
 2   Time Frame                                                                19866 non-null  object 
 3   Ind%:Construction                                                         1806 non-null   float64
 4   Ind%:Education & Health Services                                          1806 non-null   float64
 5   Ind%:Financial Activities                                                 1806 non-null   float64
 6   Ind%:Goods Producing                                          

In [163]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_IndustryEmployment_Annual_Change', conn, if_exists = 'replace', index = False)

19866